In [83]:
from lusidtools.jupyter_tools import toggle_code

"""FX Forward Pricing Models

Attributes
----------
FX Forwards
Valuation
Pricing Models
"""

toggle_code("Toggle Docstring")

# FX Forward Pricing Model Methodology

In this notebook, we create a portfolio containing a single FX forward and value the portfolio using each of the 9 applicable pricing models. Where applicable, we will produce a single net valuation for the instrument and also a valuation split on the individual cashflows using the `produce_separate_result_for_linear_otc_legs` pricing option, which we refer to as "split by legs" from now on.

For simplicity, we will demonstrate each of the models where one of the currencies in the FX forward is the same as the portfolio currency. Later, we will give an example where the reporting currency and the currencies in the forward are all distinct. In order to value the FX forward and report in a different currency, we will need to upsert three FX quotes, one for each pair.


## FX Triangulation

In order to make it explicit what market data is required in each of these examples, we will configure the market context to not attempt to infer missing FX rates from other quotes in the quote store, using the `attempt_to_infer_missing_fx=False` option in `lm.MarketOptions`.

On the other hand, when the `attempt_to_infer_missing_fx=True` market option is used, LUSID attempts to derive values for missing quotes from existing quotes, according to a set of rules which will be discussed in detail in another notebook (see [FX Triangulation Methodology](https://github.com/finbourne/sample-notebooks/blob/master/analytics/FX/fx-inference-triangulation.ipynb)). As example, given a (bid/mid/ask) quote for the currency pair GBPJPY, we can infer a (ask/mid/bid) quote for JPYGBP as `1/GBPJPY`. This is justified by an arbitrage argument whereby a trader could trade both currency pairs to make a risk free profit. Similarly, given quotes for USDGBP and USDJPY, we can infer a GBPJPY quote as `USDJPY/USDGBP`. Note that unless quotes are assumed to be at mid, the symmetry between currency pairs CCY<sub>1</sub>CCY<sub>2</sub> and CCY<sub>2</sub>CCY<sub>1</sub> is broken, so the triangulation methodology in LUSID currently assumes that all quotes are at mid.


## Market Data Curves and Interpolation

When using market data curves, such as a forward rates curve or a discounting curve, we provide a list of dates (or tenors) together with a list of values. To obtain values from the curve that do not fall on one of these specified dates or tenors, LUSID uses interpolation and extrapolation to calculate these values. In the examples below, we have demonstrated the calculations 'by hand' alongside the valuations in LUSID to explain the methodology, including extracting forward rates and discount factors from the curves using linear interpolation with no extrapolation. See the worked examples in [ForwardFromCurveUndiscounted](#2.3-forward-from-curve-undiscounted), [ForwardFromCurve](#2.4-forward-from-curve), [Reporting Currency](#3-reporting-currency) and [Forward Rates and Undiscounted Models](#4-forward-rates-and-undiscounted-models) for details. More detail on the interpolation and extrapolation methodology for FX forward curves is given in the section ["Interpolation on Forward Curves"](#5-interpolation-on-forward-curves).


## Table of Contents
1. [Create Portfolio](#1-create-portfolio)
2. [Valuation](#2-valuation)
    1. [ConstantTimeValueOfMoney](#2.1-constant-time-value-of-money)
    2. [SimpleStatic](#2.2-simple-static)
    3. [ForwardFromCurveUndiscounted](#2.3-forward-from-curve-undiscounted)
    4. [ForwardFromCurve](#2.4-forward-from-curve)
    5. [ForwardWithPointsUndiscounted](#2.5-forward-with-points-undiscounted)
    6. [ForwardWithPoints](#2.6-forward-with-points)
    7. [ForwardSpecifiedRateUndiscounted](#2.7-forward-specified-rate-undiscounted)
    8. [ForwardSpecifiedRate](#2.8-forward-specified-rate)
    9. [Discounting](#2.9-discounting)
3. [Reporting Currency](#3-reporting-currency)
4. [Forward Rates and Undiscounted Models](#4-forward-rates-and-undiscounted-models)
5. [Interpolation on Forward Curves](#5-interpolation-on-forward-curves)


**Scenario:** Consider the scenario where our company Z needs to pay a Japanese supplier 19,000,000 JPY in 6 months and we wish to hedge the currency risk and lock in a known GBP amount. To this end, today (Mon 03/06/2024) we enter into a 6 month GBPJPY forward to pay 100,000 GBP to receive 19,000,000 JPY in 6 months time (Tue 03/12/2024), corresponding to a forward rate of `19,000,000/100,000 = 190`. We will value this FX forward a few weeks prior to the maturity date, on Mon 14/10/2024.

In [84]:
# Import generic non-LUSID packages
import os
import pandas as pd
from datetime import datetime, timedelta
import pytz
from IPython.core.display import HTML
import json
import math

# Import key modules from the LUSID package
import lusid
import lusid.models as lm
from lusid.utilities import ApiClientFactory
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.4f}".format
display(HTML("<style>.container { width: 90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate user and create API client
api_factory = ApiClientFactory(api_secrets_filename=secrets_path)

display("LUSID environment initialised")
display(
    "LUSID API version:",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

'LUSID environment initialised'

'LUSID API version:'

'0.6.13528.0'

In [85]:
# Set required APIs
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
complex_market_data_api = api_factory.build(lusid.api.ComplexMarketDataApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lusid.api.AggregationApi)

## 1. Create Portfolio

We begin by creating a portfolio which will have one FX Forward holding.

In [86]:
# Define common variables
pf_scope = "fx-forward-pricing-models"
pf_code = "FxForwardPricingModels"
pf_ccy = "GBP"
pf_created_at = datetime(2024, 6, 1, tzinfo=pytz.utc)

In [87]:
# Get or create portfolio
try:
    portfolio_api.get_portfolio(scope=pf_scope, code=pf_code)
    display(f"Found portfolio {pf_code} in scope {pf_scope}")
except lusid.ApiException as e:
    body = json.loads(e.body)
    if body["name"] == "PortfolioNotFound":
        transaction_portfolios_api.create_portfolio(
            scope=pf_scope,
            create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
                display_name=pf_code,
                code=pf_code,
                base_currency=pf_ccy,
                created=pf_created_at.isoformat(),
            ),
        )

        display(f"Created new portfolio {pf_code} in scope {pf_scope}")

    else:
        display(
            f"Error fetching portfolio - Title: {body['title']}, Details: {body['detail']}, Errors: {body['errors']}"
        )

'Found portfolio FxForwardPricingModels in scope fx-forward-pricing-models'

We now create and upsert a FX forward instrument with the following characteristics:

- Buy/Receive (FGN): 19,000,000 JPY
- Sell/Pay (DOM): 100,000 GBP
- Start Date: Monday 03/06/2024
- Maturity Date: Tuesday 03/12/2024

In [88]:
forward_name = "GBPJPY 6M Forward"
forward_identifier = "FWD-GBPJPY6M"

start_date = datetime(2024, 6, 3, tzinfo=pytz.utc)
maturity_date = datetime(2024, 12, 3, tzinfo=pytz.utc)
dom_amount = -100_000
dom_ccy = "GBP"
fgn_amount = 19_000_000
fgn_ccy = "JPY"

# Create FX forward definition
fx_forward = lm.FxForward(
    start_date=start_date.isoformat(),
    maturity_date=maturity_date.isoformat(),
    dom_amount=dom_amount,
    dom_ccy=dom_ccy,
    fgn_amount=fgn_amount,
    fgn_ccy=fgn_ccy,
    instrument_type="FxForward",
)

# Create LUSID instrument definition
forward_definition = lm.InstrumentDefinition(
    name=forward_name,
    identifiers={"ClientInternal": lm.InstrumentIdValue(value=forward_identifier)},
    definition=fx_forward,
)

In [89]:
# Upsert FX forward instrument
try:
    upsert_response = instruments_api.upsert_instruments(
        request_body={forward_identifier: forward_definition}
    )
    luid = upsert_response.values[forward_identifier].lusid_instrument_id
    display(f"Success! Upserted instrument {luid}")
except KeyError as e:
    display(
        f"Failed to upsert instrument {forward_identifier}. Details: {upsert_response.failed[forward_identifier].detail}"
    )
except lusid.ApiException as e:
    body = json.loads(e.body)
    display(
        f"Title: {body['title']}, Details: {body['detail']}, Errors: {body['errors']}"
    )

'Success! Upserted instrument LUID_000185GP'

Having created the portfolio and an FX forward instrument, we now add a `StockIn` transaction to create a position of one unit of the above FX forward without incurring any costs which would affect the cash holding.

In [90]:
# Trade variables
settle_days = 2
trade_date = start_date
settlement_date = start_date + timedelta(days=settle_days)

# Create StockIn transaction
stock_in_txn = lm.TransactionRequest(
    transaction_id="TXN001",
    type="StockIn",
    instrument_identifiers={"Instrument/default/ClientInternal": forward_identifier},
    transaction_date=trade_date.isoformat(),
    settlement_date=settlement_date.isoformat(),
    units=1,
    transaction_price=lm.TransactionPrice(price=0, type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=0, currency=pf_ccy),
    exchange_rate=1,
    transaction_currency=pf_ccy,
)

# Upsert StockIn transaction
try:
    response = transaction_portfolios_api.upsert_transactions(
        scope=pf_scope, code=pf_code, transaction_request=[stock_in_txn]
    )
    display(f"Transaction successfully updated at time {response.version.as_at_date}")

except lusid.ApiException as e:
    body = json.loads(e.body)
    display(
        f"Failed to upsert transaction - Title: {body['title']}, Details: {body['detail']}, Errors: {body['errors']}"
    )

'Transaction successfully updated at time 2024-08-12 09:49:13.311039+00:00'

Before running a valuation on this portfolio, we verify the holdings are as expected - that is, the portfolio contains one unit of the above FX forward.

In [91]:
# Get holdings for portfolio
get_holdings_response = transaction_portfolios_api.get_holdings(
    scope=pf_scope,
    code=pf_code,
    # Decorate on instrument name property for clarity
    property_keys=["Instrument/default/Name"],
)

# Rename columns
column_name_mapping = {"instrument_uid": "LUID", "Name(default-Properties)": "Name"}

# Transform API response to a pandas dataframe and show it
get_holdings_response_df = lusid_response_to_data_frame(
    lusid_response=get_holdings_response,
    column_name_mapping=column_name_mapping,
    rename_properties=True,
)

ignored_columns = [
    "instrument_scope",
    "sub_holding_keys",
    "cost_portfolio_ccy.currency",
    "SourcePortfolioId(default-Properties)",
    "SourcePortfolioScope(default-Properties)",
    "notional_cost.amount",
    "notional_cost.currency",
    "settlement_schedule",
]
ignored_columns.extend(
    list(
        get_holdings_response_df.filter(
            regex="notional_cost|amortised_cost|variation_margin"
        )
    )
)

# Drop some noisy columns
get_holdings_response_df.drop(
    columns=ignored_columns,
    inplace=True,
)

display(get_holdings_response_df)

,LUID,Name,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,currency,holding_type_name,holding_id
0,LUID_000185GP,GBPJPY 6M Forward,P,1.0000,1.0000,0.0000,GBP,0.0000,GBP,Position,69113036


## 2. Valuation

Now we have created a portfolio with a holding of one unit of a GBPJPY FX forward, we will run valuation on the portfolio with each of the pricing models listed above, both split by legs and not split by legs, where applicable. 

When we choose not to split by cashflow legs, the valuation will contain a single record which shows the net present value in the domestic currency of the forward and present value in the report currency.

When we split by legs, the valuation will show a separate record for the domestic cashflow (outflow in our example) and the foreign cashflow (inflow here).

First, we'll create some helper functions for upserting pricing recipes and performing the valuation.

In [92]:
market_supplier = "Lusid"
valuation_date = datetime(2024, 10, 14, tzinfo=pytz.utc)


def create_market_context(
    scope: str, market_rules: list[lm.MarketDataKeyRule]
) -> lm.MarketContext:
    return lm.MarketContext(
        # Rules for where we should resolve rates data
        market_rules=market_rules,
        # Default options for resolving market data
        options=lm.MarketOptions(
            default_supplier=market_supplier,
            default_scope=scope,
            default_instrument_code_type="ClientInternal",
            # Do not attempt to infer missing FX rates using triangulation.
            attempt_to_infer_missing_fx=False,
        ),
    )


def upsert_recipe(recipe: lm.ConfigurationRecipe):
    recipe_request = lm.UpsertRecipeRequest(configuration_recipe=recipe)
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=recipe_request
    )


def upsert_quotes(scope: str, quotes: list[lm.UpsertQuoteRequest]):
    # Create request body object from list of quotes
    request_body = {f"{i}": x for i, x in zip(range(len(quotes)), quotes)}
    quote_response = quotes_api.upsert_quotes(scope=scope, request_body=request_body)

    if quote_response.failed == {}:
        display(
            f"Quotes successfully loaded into LUSID. {len(quote_response.values)} quotes upserted."
        )
    else:
        display(
            f"Some failures occurred during quote upsert. Failed: {quote_response.failed}"
        )


def get_daily_valuation(
    market_data_scope: str,
    date: datetime,
    recipe_code: str,
    report_currency: str = pf_ccy,
) -> pd.DataFrame:
    metrics = [
        lm.AggregateSpec(key="Instrument/CoreData/Name", op="Value"),
        lm.AggregateSpec(key="Instrument/CoreData/StartDate", op="Value"),
        lm.AggregateSpec(key="Instrument/CoreData/MaturityDate", op="Value"),
        lm.AggregateSpec(key="Valuation/LegIdentifier", op="Value"),
        lm.AggregateSpec(key="Valuation/PV", op="Value"),
        lm.AggregateSpec(key="Valuation/PV/Ccy", op="Value"),
        lm.AggregateSpec(key="Valuation/PvInReportCcy", op="Value"),
    ]

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=market_data_scope, code=recipe_code),
        metrics=metrics,
        group_by=None,
        portfolio_entity_ids=[lm.PortfolioEntityId(scope=pf_scope, code=pf_code)],
        valuation_schedule=lm.ValuationSchedule(effective_at=date.isoformat()),
        report_currency=report_currency,
    )

    valuation_data = aggregation_api.get_valuation(
        valuation_request=valuation_request
    ).data

    valuation_df = pd.DataFrame(valuation_data)
    valuation_df["Instrument/CoreData/StartDate"] = valuation_df[
        "Instrument/CoreData/StartDate"
    ].apply(lambda x: datetime.strptime(x[:10], "%Y-%m-%d").date())

    valuation_df["Instrument/CoreData/MaturityDate"] = valuation_df[
        "Instrument/CoreData/MaturityDate"
    ].apply(lambda x: datetime.strptime(x[:10], "%Y-%m-%d").date())

    # Reorder columns:
    valuation_df = valuation_df.loc[
        :,
        [
            "Instrument/CoreData/Name",
            "Instrument/CoreData/StartDate",
            "Instrument/CoreData/MaturityDate",
            "Valuation/LegIdentifier",
            "Valuation/PV",
            "Valuation/PV/Ccy",
            "Valuation/PvInReportCcy",
        ],
    ]

    return valuation_df

### 2.1 Constant Time Value of Money

The Constant Time Value of Money (CTVoM) pricing model values each of the cashflows using a spot rate on the valuation date, assuming zero discount interest rates, so no discounting is performed to calculate the present value. One implication of assuming zero discounting rates is that the implied forward rate for the maturity date is equal to the spot rate on the valuation date. For example £1 in one year is worth £1 today. Thus the present value of an FX forward under this model is simply the sum of the two amounts, which are converted to the domestic currency based on the current spot exchange rate `S`.

In this example, the GBP outflow is 100,000 GBP, while the JPY inflow of 19,000,000 JPY is worth `19,000,000 / S` in GBP. If the spot rate is 190 then the PV is zero. If the spot rate is 185, the present value is `102,702.70 - 100,000 = 2,702.70 GBP`.

In [93]:
# CTVoM valuation of FX forward
market_data_scope = "fx-forward-pricing-constant-time-value-of-money"
market_context = create_market_context(
    scope=market_data_scope,
    market_rules=[
        # Market data rule for resolving FX spot rates
        lm.MarketDataKeyRule(
            key="Fx.*.*",
            data_scope=market_data_scope,
            supplier=market_supplier,
            quote_type="Rate",
            field="mid",
        ),
    ],
)

# Not split by legs
pricing_context = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ConstantTimeValueOfMoney",
            instrument_type="FxForward",
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=False),
)

recipe_code = "ctvom-no-split"
pricing_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    description="CTVoM without split legs",
    market=market_context,
    pricing=pricing_context,
)

upsert_recipe(pricing_recipe)


# Split by legs
pricing_context_split = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ConstantTimeValueOfMoney",
            instrument_type="FxForward",
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=True),
)

recipe_code_split = "ctvom-split"
pricing_recipe_split = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code_split,
    description="CTVoM split legs",
    market=market_context,
    pricing=pricing_context_split,
)

upsert_recipe(pricing_recipe_split)

We'll now upsert a GBPJPY quote of 190, so the present value of the FX forward should be zero.

In [94]:
# Upsert market data
# First perform a valuation where the spot rate is such that the contract is worth zero.
spot_rate_gbpjpy = 190
quote_gbp_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_gbp_jpy])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

#### CTVoM No Split

Since we are assuming no discounting, the GBP outflow of 100,000 is worth 100,000 on the valuation date, and the JPY inflow of 19,000,000 is worth 19,000,000 JPY on the valuation date, which is worth `19,000,000 / 190 = 100,000` GBP at the spot rate of 190, so the present value of the forward is `-100,000 + 19,000,000 / 190 = 0` GBP.

In [95]:
# Not split by legs
get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code,
)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,None,0.0000,GBP,0.0000


#### CTVoM Split By Legs

Note that the valuation above, without split by legs, only requires a GBPJPY quote. However, to split by legs, we also require a JPYGBP quote, or we could allow the market data resolver to infer the inverse FX rate JPYGBP from the GBPJPY quote by setting `attempt_to_infer_missing_fx=True` in the market options when creating the market context.

We now upsert the inverse quote `JPYGBP = 0.00526`, which has been rounded to 5 decimal places, so there will be a small discrepancy where the values of legs in the domestic currency will not exactly sum to zero.

In [96]:
spot_rate_jpygbp = 0.00526
quote_jpy_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="JPY/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_jpygbp, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_jpy_gbp])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In [97]:
# Split by legs
get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code_split,
)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-100,000.0000",GBP,"-100,000.0000"
1,GBPJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"19,000,000.0000",JPY,"99,940.0000"


Now suppose GBP has weakened against JPY and the current GBPJPY spot rate is `S = 185 < 190`. We will now update the GBPJPY quote to 185 and the JPYGBP quote to 1/185 and re-run the above valuation.

In [98]:
# Upsert market data
spot_rate_gbpjpy = 185
quote_gbp_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

# Spot rate for conversion to report currency
spot_rate_jpygbp = 1 / spot_rate_gbpjpy
quote_jpy_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="JPY/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_jpygbp, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_gbp_jpy, quote_jpy_gbp])

'Quotes successfully loaded into LUSID. 2 quotes upserted.'

#### CTVoM No Split

As above, since we are assuming no discounting and a GBPJPY spot rate of 185, the present value of the FX forward is given by `-100,000 + 19,000,000 / 185 = 2,702.7027` GBP.

In [99]:
# Not split by legs
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code,
)

display(valuation_df)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,None,"2,702.7027",GBP,"2,702.7027"


#### CTVoM Split By Legs

As above, since we are assuming no discounting, the present value of each leg of the forward is equal to the original domestic and foreign amounts on the instrument, so the PV of the domestic leg is -100,000 GBP and the PV of the foreign leg is 19,000,000 JPY. The reporting currency in this example is GBP, so no conversion is required for the PV of the domestic leg in the report currency. The PV of the foreign leg is converted to the report currency using the JPYGBP spot rate of 1/185, giving a PV in report currency of `19,000,000 * (1/185) = 102,702.7027` GBP.

In [100]:
# Split by legs
valuation_split_df = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code_split,
)

display(valuation_split_df)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-100,000.0000",GBP,"-100,000.0000"
1,GBPJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"19,000,000.0000",JPY,"102,702.7027"


Observe that the sum of the PVs of the separate legs in the report currency equals the net valuation with `produce_separate_result_for_linear_otc_legs=False`.

In [101]:
leg_sum = valuation_split_df["Valuation/PvInReportCcy"].sum()
net_pv = valuation_split_df["Valuation/PvInReportCcy"].sum()
display(f"Sum of leg PVs: {leg_sum}")
display(f"Net PV: {net_pv}")

'Sum of leg PVs: 2702.702702702707'

'Net PV: 2702.702702702707'

### 2.2 Simple Static

The `SimpleStatic` pricing model simply performs a lookup in the quote store and returns the valuation as the value per unit (positive or negative) of the domestic currency times the absolute value of the DOM amount. This allows us to use a valuation from another source rather than deferring the valuation of a contract to LUSID. `SimpleStatic` does not support split by legs valuation since the upserted value per unit of domestic currency does not include any information that can be used to split by legs.

In this example, the DOM amount is -100,000 GBP, so we upsert a quote for the PV of the forward scaled such that we are paying 1 GBP for 190 JPY at maturity. In this case, we have valued the above forward at 0.0132 GBP (per GBP sold), so the present value of the forward for 19,000,000 JPY is `|-100,000| * 0.0132 = 1320.00 GBP` under the `SimpleStatic` pricing model.

We first create the pricing recipe.

In [102]:
# Simple Static lookup pricer valuation of FX Forwards
market_data_scope = "fx-forward-pricing-simple-static"
market_context = create_market_context(
    scope=market_data_scope,
    market_rules=[
        # Market data rule for resolving market price quotes
        lm.MarketDataKeyRule(
            key="Quote.ClientInternal.*",
            data_scope=market_data_scope,
            supplier=market_supplier,
            quote_type="Price",
            field="mid",
        )
    ],
)

pricing_context = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="SimpleStatic",
            instrument_type="FxForward",
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=False),
)

recipe_code = "simple-static"
pricing_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    description="Simple Static",
    market=market_context,
    pricing=pricing_context,
)

upsert_recipe(pricing_recipe)

Next, we upsert a quote for the value per GBP of the FX forward.

In [103]:
# Upsert market data
per_gbp_value = 0.0132
quote_fwd = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id=forward_identifier,
            instrument_id_type="ClientInternal",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=per_gbp_value, unit="GBP"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_fwd])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

Finally, we run the valuation and compare with the manual valuation described above.

In [104]:
# Perform SimpleStatic valuation
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope, date=valuation_date, recipe_code=recipe_code
)

display(valuation_df)

# By hand:
display(f"Present value: {abs(dom_amount) * per_gbp_value}")

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,None,"1,320.0000",GBP,"1,320.0000"


'Present value: 1320.0'

### 2.3 Forward From Curve Undiscounted

The `ForwardFromCurveUndiscounted` models assumes no discounting on the present value, or zero discount interest rates in both currencies (as with `ConstantTimeValueOfMoney`), so the present value of £1 at maturity is worth £1 now. The model requires a forward rates curve with base date on the valuation date and extending beyond the maturity date. The forward is valued using a forward rate quote effective on the valuation date, estimated using linear interpolation for dates or tenors not specified in the curve data.

In [105]:
def upsert_complex_market_data(scope: str, request: lm.UpsertComplexMarketDataRequest):
    response = complex_market_data_api.upsert_complex_market_data(
        scope=scope, request_body={"1": request}
    )

    if response.failed == {}:
        display(
            f"Quotes successfully loaded into LUSID. {len(response.values)} quotes upserted."
        )
    else:
        display(
            f"Some failures occurred during quote upsert. Failed: {response.failed}"
        )

In [106]:
# Create market context
market_data_scope = "fx-forward-pricing-curve-undiscounted"
market_context = create_market_context(
    scope=market_data_scope,
    market_rules=[
        # Market data rule for resolving FX spot rates
        lm.MarketDataKeyRule(
            key="Fx.*.*",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
            quote_interval="5D.0D",
        ),
        # Market data rule for resolving forward rate curves from complex market data store.
        lm.MarketDataKeyRule(
            key="FxForwards.*.*.FxFwdCurve",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
            quote_interval="1Y.0D",
        ),
    ],
)

# Not split by legs
pricing_context = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardFromCurveUndiscounted",
            instrument_type="FxForward",
        )
    ]
)

recipe_code = "curve-undiscounted-no-split"
pricing_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    description="FX forward curve, no discounting",
    market=market_context,
    pricing=pricing_context,
)

upsert_recipe(pricing_recipe)


# Split by legs
pricing_context_split = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardFromCurveUndiscounted",
            instrument_type="FxForward",
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=True),
)

recipe_code_split = "curve-undiscounted-split"
pricing_recipe_split = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code_split,
    description="FX forward curve, no discounting, split by legs",
    market=market_context,
    pricing=pricing_context_split,
)

upsert_recipe(pricing_recipe_split)

Now we create and upsert a GBPJPY forward rates tenor curve and a GBPJPY spot rate quote. 

If doing a net valuation (not split by legs) and not converting to report currency, a spot GBPJPY quote is still required but its value does not impact the net valuation, since forward rates from the curve are used. However, it is used to value the foreign leg in the foreign currency, so it is relevant for the split by legs valuation.

If converting to report currency (RPT), this spot rate is not used at all since everything is priced using forward rates taken from forward curves for RPTDOM and RPTFGN currency pairs.

In [107]:
# Upsert market data
# FX forward tenor curve
forward_curve_data = lm.FxForwardTenorCurveData(
    market_data_type="FxForwardTenorCurveData",
    base_date=valuation_date.isoformat(),
    dom_ccy=dom_ccy,
    fgn_ccy=fgn_ccy,
    tenors=["1W", "2W", "1M", "2M", "3M"],
    rates=[188.32, 188.14, 187.67, 186.93, 186.11],
)

fwd_curve_id = lm.ComplexMarketDataId(
    provider="Lusid",
    price_source=None,
    effective_at=valuation_date.isoformat(),
    market_asset="GBP/JPY/FxFwdCurve",
)

complex_market_data_request = lm.UpsertComplexMarketDataRequest(
    market_data_id=fwd_curve_id,
    market_data=forward_curve_data,
)

# Upsert forward curve into LUSID
response = upsert_complex_market_data(
    scope=market_data_scope, request=complex_market_data_request
)

# GBPJPY spot rate
spot_rate_gbpjpy = 188.5
quote_gbp_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_gbp_jpy])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

Before running the valuations through LUSID, we will work out the example manually to illustrate the methodology. The justification behind the methodology is based on entering into an opposing forward on the valuation date which neutralises the foreign cash flow on maturity, leading to a net domestic cashflow, which is taken as the present value of the original forward contract on the valuation date.

Since we are assuming no discounting, the value of the domestic leg is `dom_amount = -100,000 GBP`.

To calculate the present value of the foreign leg in the domestic currency, we need to estimate the forward rate for a (hypothetical) forward starting on the valuation date (Mon 14/10/24) and with the same maturity date (Tue 03/12/24) using linear interpolation on the forward curve we have created. The maturity date falls between the 1M and 2M tenors, so now we need to work out the weightings for linear interpolation.

**1M:** Valuation is on Monday 14/10/24, spot date (+2) is Wednesday 16/10/24. Increment by 1 month and roll forwards over the weekend to Monday 18/11/24. This is 15 calendar days before maturity.

**2M:** Valuation is on Monday 14/10/24, spot date (+2) is Wednesday 16/10/24. Increment by 2 months to Monday 16/12/24. This is 13 calendar days after maturity.

The period between the 1M and 2M tenors spans 28 days. The forward rate on the valuation date (Mon 14/10/24) implied by linear interpolation on the curve between the 1M and 2M tenors is: `f = f(1M) + (15/28) * (f(2M) - f(1M))`, which can be rearranged as `f = ((28 - 15)/28) * f(1M) + (15/28) * f(2M) = 13 * f(1M) + 15 * f(2M)`.

In [108]:
def manual_valuation():
    # Domestic leg PV
    dom_pv = -100_000

    # Estimate forward rate from curve:
    fwd_1m = 187.67
    fwd_2m = 186.93
    fwd_interpolated = (13 * fwd_1m + 15 * fwd_2m) / 28
    display(f"Interpolated forward rate: {fwd_interpolated}")

    # Foreign leg PV in domestic currency
    fgn_pv = 19_000_000 / fwd_interpolated
    net_pv = dom_pv + fgn_pv
    display(f"Net present value: {net_pv}")

    # Foreign leg price in FGN
    spot = 188.5
    fgn_leg_price = fgn_pv * spot
    display(f"Foreign leg PV in FGN: {fgn_leg_price}")


manual_valuation()

'Interpolated forward rate: 187.27357142857142'

'Net present value: 1455.8533543364756'

'Foreign leg PV in FGN: 19124428.357292425'

#### FX Forward Curve Undiscounted - No Split

The calculation above illustrates the methodology in LUSID for arriving at the net present value of the FX forward using a forward rates curve and a spot GBPJPY quote.

In [109]:
# Not split by legs
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope, date=valuation_date, recipe_code=recipe_code
)

display(valuation_df)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,None,"1,455.8534",GBP,"1,455.8534"


#### FX Forward Curve Undiscounted - Split By Legs

As illustrated above, to produce separate results for each leg, we now require a JPYGBP quote to convert the PV of the foreign leg into the reporting currency GBP.

In [110]:
spot_rate_jpygbp = 0.00531
quote_jpy_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="JPY/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_jpygbp, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_jpy_gbp])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In [111]:
valuation_split_df = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code_split,
)

display(valuation_split_df)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-100,000.0000",GBP,"-100,000.0000"
1,GBPJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"19,124,428.3573",JPY,"101,550.7146"


Note that the sum of the PVs in the report currency of the two legs does not exactly equal the net valuation above. This is due to rounding on the inverse JPYGBP quote and leads to a small difference relative to the notional. LUSID does not enforce the consistency of market data, so we need to take care that market data is reasonable.

In [112]:
# GBPJPY * JPYGBP != 1
display(f"GBPJPY * JPYGBP = {188.5 * 0.00531}")
display(f"Actual inverse rate 1/GBPJPY = {1 / 188.5}")

'GBPJPY * JPYGBP = 1.000935'

'Actual inverse rate 1/GBPJPY = 0.005305039787798408'

We now show that the net PV derived from the sum of the two leg PVs in report currency is consistent with the net valuation based on the sum of the leg PVs in the leg currency, converted to GBP at the spot rate `JPYGBP = 0.00531`.

In [113]:
# Net value derived from the sum of the two legs
leg_sum = valuation_split_df["Valuation/PvInReportCcy"].sum()
display(f"Sum of leg PVs in report CCY = {leg_sum}")

# Valuation based on spot rate JPYGBP = 0.00531 is consistent with the sum of leg PVs
display(19_124_428.3573 * 0.00531 - 100_000)

'Sum of leg PVs in report CCY = 1550.7145772230142'

1550.7145772629883

We now show that the sum of the leg PVs in the leg currency, converted at the exact inverse rate `JPYGBP = 1/185` is consistent with the net valuation with the `produce_separate_result_for_linear_otc_legs=True` option, so the apparent discrepancy is purely due to rounding on the inverse quote JPYGBP.

In [114]:
# Using the exact inverse rate without rounding leads to the same result as net the valuation
net_value = valuation_df["Valuation/PvInReportCcy"].sum()
display(f"Net PV in report currency = {net_value}")
display(19_124_428.3573 * (1 / 188.5) - 100_000)

'Net PV in report currency = 1455.853354336707'

1455.853354376639

### 2.4 Forward From Curve

The `ForwardFromCurve` pricing model is the same as the `ForwardFromCurveUndiscounted` model, except that discounting is applied to future values to derive the present value. This model requires a discount factor curve as well as the forward rates curve required above.

In [115]:
# Create market context
market_data_scope = "fx-forward-pricing-curve"
market_context = create_market_context(
    scope=market_data_scope,
    market_rules=[
        # Market data rule for resolving FX spot rates
        lm.MarketDataKeyRule(
            key="Fx.*.*",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
            quote_interval="5D.0D",
        ),
        # Market data rule for resolving forward rates curves
        lm.MarketDataKeyRule(
            key="FxForwards.*.*.FxFwdCurve",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
            quote_interval="1Y.0D",
        ),
        # Market data rule for resolving discount factor curves
        lm.MarketDataKeyRule(
            key="Rates.GBP.GBPOIS",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
        ),
    ],
)

# Not split by legs
pricing_context = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardFromCurve",
            instrument_type="FxForward",
            model_options=lm.FxForwardModelOptions(
                model_options_type="FxForwardModelOptions",
                forward_rate_observable_type="FxForwardCurve",
                discounting_method="Standard",
                convert_to_report_ccy=False,
            ),
        )
    ],
)

recipe_code = "curve-no-split"
pricing_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    description="FX forward curve, with discounting",
    market=market_context,
    pricing=pricing_context,
)

upsert_recipe(pricing_recipe)


# Split by legs
pricing_context_split = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardFromCurve",
            instrument_type="FxForward",
            model_options=lm.FxForwardModelOptions(
                model_options_type="FxForwardModelOptions",
                forward_rate_observable_type="FxForwardCurve",
                discounting_method="Standard",
                convert_to_report_ccy=False,
            ),
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=True),
)

recipe_code_split = "curve-split"
pricing_recipe_split = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code_split,
    description="FX forward curve, with discounting, split by legs",
    market=market_context,
    pricing=pricing_context_split,
)

upsert_recipe(pricing_recipe_split)

We now create and upsert a GBPJPY forward rates curve, a GBP discount factor curve and a GBPJPY spot rate quote.

If doing a net valuation (not split by legs) and not converting to report currency, a spot GBPJPY quote is still required but its value does not impact the net valuation, since forward rates from the curve are used. However, it is used to value the foreign leg in the foreign currency, so it is relevant for the split by legs valuation.

If converting to report currency (RPT), this spot rate is not used at all since everything is priced using forward rates taken from forward curves for RPTDOM and RPTFGN currency pairs.

In [116]:
# Upsert market data
# FX forward tenor curve
forward_curve_data = lm.FxForwardTenorCurveData(
    base_date=valuation_date.isoformat(),
    dom_ccy=dom_ccy,
    fgn_ccy=fgn_ccy,
    tenors=["1W", "2W", "1M", "2M", "3M"],
    rates=[188.32, 188.14, 187.67, 186.93, 186.11],
    market_data_type="FxForwardTenorCurveData",
)

fwd_curve_id = lm.ComplexMarketDataId(
    provider="Lusid",
    price_source=None,
    effective_at=valuation_date.isoformat(),
    market_asset="GBP/JPY/FxFwdCurve",
)

fwd_curve_request = lm.UpsertComplexMarketDataRequest(
    market_data_id=fwd_curve_id,
    market_data=forward_curve_data,
)

# Upsert forward curve into LUSID
upsert_complex_market_data(scope=market_data_scope, request=fwd_curve_request)


# DOM currency discount factor curve
# Build curve data
df_months = 6
rfr = 0.05
df_dates = [None] * (df_months - 1)
dfs = [None] * (df_months - 1)
for i in range(1, df_months):
    df_dates[i - 1] = (valuation_date + timedelta(days=i * 30)).isoformat()
    dfs[i - 1] = math.exp(-30 / 360 * rfr * i)


df_curve_data = lm.DiscountFactorCurveData(
    base_date=valuation_date.isoformat(),
    dates=df_dates,
    discount_factors=dfs,
    market_data_type="DiscountFactorCurveData",
)

df_curve_id = lm.ComplexMarketDataId(
    provider="Lusid",
    price_source="",
    effective_at=valuation_date.isoformat(),
    market_asset="GBP/GBPOIS",
)

df_curve_request = lm.UpsertComplexMarketDataRequest(
    market_data_id=df_curve_id, market_data=df_curve_data
)

upsert_complex_market_data(scope=market_data_scope, request=df_curve_request)


# GBPJPY spot rate
spot_rate_gbpjpy = 188.5
quote_gbp_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_gbp_jpy])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

Before running the valuations through LUSID, we will work out the example manually to illustrate the methodology. The first step is to estimate the discount factor in the domestic currency to calculate the present value of the domestic leg. Again, this will be estimated using linear interpolation on the points of the discount curve.

Next, the present value of the foreign leg in the domestic currency is calculated by first dividing by the forward rate estimated from the curve to get the future value of the foreign leg in DOM, then applying the discount factor to get the present value.

Finally, the present values of the two legs are added to derive the net present value of the forward.


The points on the discount factor curve we have upserted are at +30 day increments from the valuation date (Mon 14/10/24), so the maturity date falls between the first two points of the curve; the +30 and +60 day increments, which are Wed 13/11/24 and Fri 13/12/24 respectively. Thus there are 20 days from +30 to maturity and 10 days from maturity to +60, so the estimated discount factor will be the 1:2 weighted average of the +30 and +60 day discount factors.


As above, to estimate the forward rate for a (hypothetical) forward starting on the valuation date (Mon 14/10/24) and with the same maturity date (Tue 03/12/24) using linear interpolation on the forward curve we have created. The maturity date falls between the 1M and 2M tenors, so now we need to work out the weightings for linear interpolation.

**1M:** Valuation is on Monday 14/10/24, spot date (+2) is Wednesday 16/10/24. Increment by 1 month and roll forwards over the weekend to Monday 18/11/24. This is 15 calendar days before maturity.

**2M:** Valuation is on Monday 14/10/24, spot date (+2) is Wednesday 16/10/24. Increment by 2 months to Monday 16/12/24. This is 13 calendar days after maturity.

The period between the 1M and 2M tenors spans 28 days. The forward rate on the valuation date (Mon 14/10/24) implied by linear interpolation on the curve between the 1M and 2M tenors is: `f = f(1M) + (15/28) * (f(2M) - f(1M))`, which can be rearranged as `f = ((28 - 15)/28) * f(1M) + (15/28) * f(2M) = 13 * f(1M) + 15 * f(2M)`.

In [117]:
def manual_valuation():
    # Estimate DOM discount factor using linear interpolation:
    rfr = 0.05
    df_1m = math.exp(-30 / 360 * rfr * 1)
    df_2m = math.exp(-30 / 360 * rfr * 2)
    plus30 = valuation_date + timedelta(days=30)
    plus60 = valuation_date + timedelta(days=60)
    d1 = (maturity_date - plus30).days
    d2 = (plus60 - maturity_date).days
    df_interpolated = (d2 * df_1m + d1 * df_2m) / (d1 + d2)
    display(f"Interpolated discount factor: {df_interpolated}")

    # Estimate forward rate from curve
    fwd_1m = 187.67
    fwd_2m = 186.93
    fwd_interpolated = (13 * fwd_1m + 15 * fwd_2m) / 28
    display(f"Interpolated forward rate: {fwd_interpolated}")

    # Domestic leg PV
    dom_pv = -100_000 * df_interpolated
    display(f"Domestic leg PV: {dom_pv}")

    # Foreign leg PV in DOM
    fgn_pv = (19_000_000 * df_interpolated) / fwd_interpolated
    net_pv = dom_pv + fgn_pv
    display(f"Foreign leg PV in DOM: {fgn_pv}")
    display(f"Net present value: {net_pv}")

    # Foreign leg PV in FGN:
    spot = 188.5
    fgn_leg_price = fgn_pv * spot
    display(f"Foreign leg PV in FGN: {fgn_leg_price}")

    # The FGN discount factor implied by the DOM discount, assuming no arbitrage opportunities
    implied_fgn_df = df_interpolated * spot / fwd_interpolated
    display(f"Implied FGN discount factor: {implied_fgn_df}")


manual_valuation()

'Interpolated discount factor: 0.9930815290409539'

'Interpolated forward rate: 187.27357142857142'

'Domestic leg PV: -99308.15290409539'

'Foreign leg PV in DOM: 100753.93397927926'

'Net present value: 1445.7810751838697'

'Foreign leg PV in FGN: 18992116.55509414'

'Implied FGN discount factor: 0.9995850818470602'

#### FX Forward Curve - No Split

The result below shows the valuation of the FX forward in LUSID using a GBPJPY forward rates curve, a GBP discount factor curve and a GBPJPY spot rate. The calculation is explained above.

In [118]:
# Not split by legs
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope, date=valuation_date, recipe_code=recipe_code
)

display(valuation_df)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,None,"1,445.7783",GBP,"1,445.7783"


#### FX Forward Curve - Split By Legs

The result below shows the valuation of the FX forward in LUSID using a GBPJPY forward rates curve, a GBP discount factor curve and a GBPJPY spot rate. To produce separate results for each leg, we require a JPYGBP spot rate quote to convert the PV of the foreign leg to the report currency GBP. The calculation is demonstrated above.

In [119]:
spot_rate_jpygbp = 0.00531
quote_jpy_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="JPY/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_jpygbp, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_jpy_gbp])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In [120]:
valuation_split_df = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code_split,
)

display(valuation_split_df)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-99,307.9612",GBP,"-99,307.9612"
1,GBPJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"18,992,079.9022",JPY,"100,847.9443"


As in the forward curve undiscounted example, note that the sum of the PVs in report currency of the two legs does not equal the net valuation above. This is purely due to rounding on the inverse quote JPYGBP and the difference in the valuations is small relative to the notional of the forward. Here we have the freedom to upsert inconsistent quotes, so we need to take care that market data is valid and reasonable.

In [121]:
# Net value derived from the sum of the two legs
leg_sum = valuation_split_df["Valuation/PvInReportCcy"].sum()
display(f"Sum of leg PVs in report CCY = {leg_sum}")

# Valuation based on spot rate JPYGBP = 0.00571 is consistent with the sum of leg PVs
display(18_992_079.9022 * 0.00531 - 99_307.9612)

'Sum of leg PVs in report CCY = 1539.9830314317078'

1539.983080681981

In [122]:
# Using the exact inverse rate leads to the same result as the net valuation.
net_value = valuation_df["Valuation/PvInReportCcy"].sum()
display(f"Net PV in report CCY = {net_value}")
display(18_992_079.9022 * (1 / 188.5) - 99_307.9612)

'Net PV in report CCY = 1445.7782849674238'

1445.7783342174953

### 2.5 Forward With Points Undiscounted

The `ForwardWithPointsUndiscounted` pricer uses the spot rate and a forward points quote to calculate a forward rate, which is used as in the other models to value the FX forward. This is an undiscounted model, so the present value of £1 in the future is always £1. The same results can be achieved by using a constant forward points curve with the `ForwardFromCurveUndiscounted` model.

We now explain the methodology used to value an FX forward for a currency pair DOMFGN using the `ForwardWithPointsUndiscounted` model, assuming `convert_to_report_ccy=False`. Let `Cf` and `Cd` denote the foreign and domestic amounts on the forward respectively. Assume we are given a spot rate `DOMFGN = S` and a number of forward points `p`. The effective forward rate is calculated as `F = S + p / 10,000`. The PV of the domestic leg in the domestic currency is `Cd`, since we are assuming no discounting. The PV of the foreign leg in the domestic currency is given by dividing the foreign amount by the forward rate `F`, so `Fgn leg PV in DOM = Cf / F = Cf / (S + p / 10,000)`. The net present value of the forward is the sum of the PVs of the legs in the domestic currency, so `net PV in DOM = Cd + Cf / F`. The PV of the foreign leg in the foreign currency is converted from the leg PV in the domestic currency at the spot rate, so `Fgn leg PV in FGN = (Cf / F) * S = Cf * S / (S + p / 10,000)`. This calculation is shown below with the same FX forward definition and market data as in the valuation performed through LUSID below.

In [123]:
def manual_valuation():
    dom_amt = -100_000
    fgn_amt = 19_000_000
    fwd_points = -108 * 100
    spot_gbpjpy = 191.11

    # Calculate forward rate from spot rate and forward points.
    effective_fwd_rate = spot_gbpjpy + fwd_points / 10_000
    display(f"Effective forward rate GBPJPY: {effective_fwd_rate}")

    # Calculate individual leg PVs in GBP
    dom_pv = dom_amt
    fgn_pv_in_gbp = fgn_amt / effective_fwd_rate
    display(f"Domestic leg PV (GBP): {dom_pv}")
    display(f"Foreign leg PV (GBP): {fgn_pv_in_gbp}")

    # Calculate foreign leg PV in the foreign currency, converted at spot.
    fgn_pv_in_jpy = fgn_pv_in_gbp * spot_gbpjpy
    display(f"Foreign PV (JPY): {fgn_pv_in_jpy}")

    # Calculate net PV of the forward in GBP
    net_pv = dom_pv + fgn_pv_in_gbp
    display(f"Net PV (GBP): {net_pv}")


manual_valuation()

'Effective forward rate GBPJPY: 190.03'

'Domestic leg PV (GBP): -100000'

'Foreign leg PV (GBP): 99984.213018997'

'Foreign PV (JPY): 19107982.95006052'

'Net PV (GBP): -15.786981002995162'

Now create the market data context and valuation recipes to perform net and individual leg valuations.

In [124]:
# Create market context
market_data_scope = "fx-forward-pricing-fwd-points-undiscounted"
market_context = create_market_context(
    scope=market_data_scope,
    market_rules=[
        # Market data rule for resolving FX spot rates
        lm.MarketDataKeyRule(
            key="Fx.*.*",
            data_scope=market_data_scope,
            supplier=market_supplier,
            quote_type="Rate",
            field="mid",
        ),
        # Market data rule for resolving forward points quote
        lm.MarketDataKeyRule(
            key="Quote.LusidInstrumentId.*",
            data_scope=market_data_scope,
            supplier=market_supplier,
            quote_type="Price",
            field="mid",
        ),
    ],
)

# Not split by legs
pricing_context = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardWithPointsUndiscounted",
            instrument_type="FxForward",
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=False),
)

recipe_code = "forward-points-undiscounted-no-split"
pricing_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    description="Forward with points undiscounted",
    market=market_context,
    pricing=pricing_context,
)

upsert_recipe(pricing_recipe)


# Split by legs
pricing_context_split = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardWithPointsUndiscounted",
            instrument_type="FxForward",
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=True),
)

recipe_code_split = "forward-points-undiscounted-split"
pricing_recipe_split = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code_split,
    description="Forward points undiscounted, split by legs",
    market=market_context,
    pricing=pricing_context_split,
)

upsert_recipe(pricing_recipe_split)

Now we upsert a spot rate quote and a forward points quote to run a single line valuation. 

In [125]:
fwd_points_gbpjpy = -108 * 100  # Scale factor of 100 required for JPY.
quote_fwd_points_gbpjpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY/Points/20241203",
            instrument_id_type="LusidInstrumentId",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=fwd_points_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

spot_rate_gbpjpy = 191.11
quote_gbp_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_fwd_points_gbpjpy, quote_gbp_jpy])

'Quotes successfully loaded into LUSID. 2 quotes upserted.'

#### Forward With Points Undiscounted - No Split

Now we obtain a net valuation of the FX forward through LUSID. Observe that the PV is the same as given the the manual calculation at the beginning of this example.

In [126]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code,
)

valuation_df

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,None,-15.7870,GBP,-15.7870


#### Forward With Points Undiscounted - Split By Legs

Now we use the pricing recipe with the `produce_separate_result_for_linear_otc_legs=True` option to run a valuation with a separate valuation for each leg of the FX forward. In order to report the PV of the foreign leg in the foreign currency, we require a JPYGBP spot rate. The PV of each leg displayed in the table below is the same as given by the manual calculation at the beginning of this example.

In [127]:
spot_rate_jpygbp = 1 / 191.11
quote_jpy_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="JPY/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_jpygbp, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_jpy_gbp])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In [128]:
valuation_df_split = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code_split,
)

valuation_df_split

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-100,000.0000",GBP,"-100,000.0000"
1,GBPJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"19,107,982.9501",JPY,"99,984.2130"


### 2.6 Forward With Points

The `ForwardWithPoints` pricer uses the spot rate and a forward points quote to calculate a forward rate, which is used as in the other models to value the FX forward, along with a discount rates curve. The same results can be achieved by using a constant forward points curve with the `ForwardFromCurve` model.

We now explain the methodology used to value an FX forward for a currency pair DOMFGN using the `ForwardWithPoints` model, assuming `convert_to_report_ccy=False`. Let `Cf` and `Cd` denote the foreign and domestic amounts on the forward respectively. Assume we are given a spot rate `DOMFGN = S` and a number of forward points `p`. This model also requires a discount factor curve in the domestic currency and we denote the discount factor for the maturity date by `D`. The expressions used to calculate the results are shown in the table below:

| Result             | Expression         |
| ------------------ | ------------------ |
| DfDom              | D                  |
| FxFwdRate          | F = S + p / 10,000 |
| DomLegPrice (DOM)  | Cd * D             |
| FGN leg PV in DOM* | (Cf / F) * D       |
| PV (DOM)           | (Cd + Cf / F) * D  |
| FgnLegPrice (FGN)  | (Cf / F) * D * S   |
| DfFgn              | D * S / F          |

_*Intermediate result not returned to the user._

In [129]:
# Create market context
market_data_scope = "fx-forward-pricing-fwd-points"
market_context = create_market_context(
    scope=market_data_scope,
    market_rules=[
        # Market data rule for resolving FX spot rates
        lm.MarketDataKeyRule(
            key="Fx.*.*",
            data_scope=market_data_scope,
            supplier=market_supplier,
            quote_type="Rate",
            field="mid",
        ),
        # Market data rule for resolving points quote
        lm.MarketDataKeyRule(
            key="Quote.LusidInstrumentId.*",
            data_scope=market_data_scope,
            supplier=market_supplier,
            quote_type="Price",
            field="mid",
        ),
        # Market data rule for resolving discount factor curves
        lm.MarketDataKeyRule(
            key="Rates.GBP.GBPOIS",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
        ),
    ],
)

# Not split by legs
pricing_context = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardWithPoints",
            instrument_type="FxForward",
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=False),
)

recipe_code = "forward-points-no-split"
pricing_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    description="Forward with points",
    market=market_context,
    pricing=pricing_context,
)

upsert_recipe(pricing_recipe)


# Split by legs
pricing_context_split = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardWithPoints",
            instrument_type="FxForward",
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=True),
)

recipe_code_split = "forward-points-split"
pricing_recipe_split = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code_split,
    description="Forward points, split by legs",
    market=market_context,
    pricing=pricing_context_split,
)

upsert_recipe(pricing_recipe_split)

Now we upsert a spot rate quote and a forward points quote to run a single line valuation. 

In [130]:
fwd_points_gbpjpy = -108 * 100  # Scale factor of 100 required for JPY.
quote_fwd_points_gbpjpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY/Points/20241203",
            instrument_id_type="LusidInstrumentId",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=fwd_points_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

spot_rate_gbpjpy = 191.11
quote_gbp_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_fwd_points_gbpjpy, quote_gbp_jpy])

'Quotes successfully loaded into LUSID. 2 quotes upserted.'

Now we require a GBP discount curve.

In [131]:
# DOM currency discount factor curve
# Build curve data
df_months = 6
rfr = 0.05
df_dates = [None] * (df_months - 1)
dfs = [None] * (df_months - 1)
for i in range(1, df_months):
    df_dates[i - 1] = (valuation_date + timedelta(days=i * 30)).isoformat()
    dfs[i - 1] = math.exp(-30 / 360 * rfr * i)


df_curve_data = lm.DiscountFactorCurveData(
    base_date=valuation_date.isoformat(),
    dates=df_dates,
    discount_factors=dfs,
    market_data_type="DiscountFactorCurveData",
)

df_curve_id = lm.ComplexMarketDataId(
    provider="Lusid",
    price_source="",
    effective_at=valuation_date.isoformat(),
    market_asset="GBP/GBPOIS",
)

df_curve_request = lm.UpsertComplexMarketDataRequest(
    market_data_id=df_curve_id, market_data=df_curve_data
)

upsert_complex_market_data(scope=market_data_scope, request=df_curve_request)

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

We now demonstrate the valuation explicitly, using the data from the economic definition of the FX forward and the market data provided above. The methodology is similar to the `ForwardWithPointsUndiscounted` example above, except that we need to calculate the GBP discount factor from the discount factor curve created above. The first step is to estimate the discount factor in the domestic currency to calculate the present value of the domestic leg. This will be estimated using linear interpolation on the points of the discount curve.

The points on the discount factor curve we have upserted are at +30 day increments from the valuation date (Mon 14/10/24), so the maturity date falls between the first two points of the curve; the +30 and +60 day increments, which are Wed 13/11/24 and Fri 13/12/24 respectively. Thus there are 20 days from +30 to maturity and 10 days from maturity to +60, so the estimated discount factor will be the 1:2 weighted average of the +30 and +60 day discount factors.

In [132]:
def manual_valuation():
    # Estimate DOM discount factor using linear interpolation:
    rfr = 0.05
    df_1m = math.exp(-30 / 360 * rfr * 1)
    df_2m = math.exp(-30 / 360 * rfr * 2)
    plus30 = valuation_date + timedelta(days=30)
    plus60 = valuation_date + timedelta(days=60)
    d1 = (maturity_date - plus30).days
    d2 = (plus60 - maturity_date).days
    df_interpolated = (d2 * df_1m + d1 * df_2m) / (d1 + d2)
    display(f"Interpolated discount factor: {df_interpolated}")

    dom_amt = -100_000
    fgn_amt = 19_000_000
    fwd_points = -108 * 100
    spot_gbpjpy = 191.11

    # Calculate forward rate from spot rate and forward points.
    effective_fwd_rate = spot_gbpjpy + fwd_points / 10_000
    display(f"Effective forward rate GBPJPY: {effective_fwd_rate}")

    # Calculate individual leg PVs in GBP
    dom_pv = dom_amt * df_interpolated
    fgn_pv_in_gbp = (fgn_amt / effective_fwd_rate) * df_interpolated
    display(f"Domestic leg PV (GBP): {dom_pv}")
    display(f"Foreign leg PV (GBP): {fgn_pv_in_gbp}")

    # Calculate foreign leg PV in the foreign currency, converted at spot.
    fgn_pv_in_jpy = fgn_pv_in_gbp * spot_gbpjpy
    display(f"Foreign PV (JPY): {fgn_pv_in_jpy}")

    # Calculate net PV of the forward in GBP
    net_pv = dom_pv + fgn_pv_in_gbp
    display(f"Net PV (GBP): {net_pv}")


manual_valuation()

'Interpolated discount factor: 0.9930815290409539'

'Effective forward rate GBPJPY: 190.03'

'Domestic leg PV (GBP): -99308.15290409539'

'Foreign leg PV (GBP): 99292.475144862'

'Foreign PV (JPY): 18975784.924934577'

'Net PV (GBP): -15.6777592333965'

#### Forward With Points - No Split

In [133]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code,
)

valuation_df

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,None,-15.6777,GBP,-15.6777


#### Forward With Points - Split By Legs

We now create and upsert a JPYGBP spot rate quote, which is required to convert the PV of the foreign leg back to the report currency GBP.

In [134]:
spot_rate_jpygbp = 1 / 191.11
quote_jpy_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="JPY/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_jpygbp, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_jpy_gbp])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In [135]:
valuation_df_split = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code_split,
)

valuation_df_split

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-99,307.9612",GBP,"-99,307.9612"
1,GBPJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"18,975,748.3035",JPY,"99,292.2835"


### 2.7 Forward Specified Rate Undiscounted

The `ForwardSpecifiedRateUndiscounted` pricer uses an FX forward rate quote along with a spot rate quote to value the FX forward. This is an undiscounted model, so the present value of £1 in the future is always £1. The same results can be achieved by using a constant forward rates curve with the `ForwardFromCurveUndiscounted` model.

In [136]:
# Create market context
market_data_scope = "fx-forward-pricing-specified-rate-undiscounted"
market_context = create_market_context(
    scope=market_data_scope,
    market_rules=[
        # Market data rule for resolving FX spot rates
        lm.MarketDataKeyRule(
            key="Fx.*.*",
            data_scope=market_data_scope,
            supplier=market_supplier,
            quote_type="Rate",
            field="mid",
            quote_interval="5D.0D",
        ),
        # Market data rule for resolving forward rate quote
        lm.MarketDataKeyRule(
            key="Quote.LusidInstrumentId.*",
            data_scope=market_data_scope,
            supplier=market_supplier,
            quote_type="Price",
            field="mid",
        ),
    ],
)

# Not split by legs
pricing_context = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardSpecifiedRateUndiscounted",
            instrument_type="FxForward",
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=False),
)

recipe_code = "forward-rate-undiscounted-no-split"
pricing_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    description="Forward specified rate undiscounted",
    market=market_context,
    pricing=pricing_context,
)

upsert_recipe(pricing_recipe)


# Split by legs
pricing_context_split = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardSpecifiedRateUndiscounted",
            instrument_type="FxForward",
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=True),
)

recipe_code_split = "forward-rate-undiscounted-split"
pricing_recipe_split = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code_split,
    description="Forward specified rate undiscounted, split by legs",
    market=market_context,
    pricing=pricing_context_split,
)

upsert_recipe(pricing_recipe_split)

Now we upsert a forward rate quote and a spot rate quote for GBPJPY to run the single line valuation.

In [137]:
fwd_rate_gbpjpy = 190.03
quote_fwd_gbpjpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY/FxFwdRate/20241203",
            instrument_id_type="LusidInstrumentId",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=fwd_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

spot_rate_gbpjpy = 191.11
quote_gbp_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_fwd_gbpjpy, quote_gbp_jpy])

'Quotes successfully loaded into LUSID. 2 quotes upserted.'

We now demonstrate the valuation explicitly, using the specified forward rate quote created above, assuming no discounting. Since we are assuming no discounting, the PV of the domestic leg is still `-100,000 GBP`. Given a GBPJPY forward rate of `F = 190.03` for maturity date 03/12/2024 effective on the valuation date, the PV of the foreign leg in GBP is `19,000,000 / F = 19,000,000 / 190.03 = 99984.2130 GBP`, which is then converted at the GBPJPY spot rate `S = 191.11` to give the PV of the foreign leg in JPY as `(19,000,000 / F) * S = (19,000,000 / 190.03) * 191.11 = 19107982.9501 JPY`. The net PV is given by the sum of the leg PVs in GBP, which is `-100,000 + 19,000,000 / 190.03 = -15.7870 GBP`. 

In [138]:
def manual_valuation():
    fwd_rate = 190.03
    dom_amt = -100_000
    fgn_amt = 19_000_000
    spot_rate = 191.11

    dom_leg_pv_gbp = dom_amt
    fgn_leg_pv_gbp = fgn_amt / fwd_rate
    net_pv_gbp = dom_leg_pv_gbp + fgn_leg_pv_gbp
    display(f"Domestic leg PV (GBP): {dom_leg_pv_gbp}")
    display(f"Foreign leg PV (GBP): {fgn_leg_pv_gbp}")
    display(f"Net PV (GBP): {net_pv_gbp}")

    fgn_leg_pv_jpy = fgn_leg_pv_gbp * spot_rate
    display(f"Foreign leg PV (JPY): {fgn_leg_pv_jpy}")


manual_valuation()

'Domestic leg PV (GBP): -100000'

'Foreign leg PV (GBP): 99984.213018997'

'Net PV (GBP): -15.786981002995162'

'Foreign leg PV (JPY): 19107982.95006052'

#### Forward Specified Rate Undiscounted - No Split

In [139]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code,
)

valuation_df

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,None,-15.7870,GBP,-15.7870


#### Forward Specified Rate Undiscounted - Split By Legs

In [140]:
spot_rate_jpygbp = 1 / 191.11
quote_jpy_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="JPY/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_jpygbp, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_jpy_gbp])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In [141]:
valuation_df_split = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code_split,
)

valuation_df_split

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-100,000.0000",GBP,"-100,000.0000"
1,GBPJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"19,107,982.9501",JPY,"99,984.2130"


### 2.8 Forward Specified Rate

The `ForwardSpecifiedRate` pricer uses an FX forward rate quote, a spot rate quote and a discount rate curve to value the FX forward. The same results can be achieved by using a constant forward rates curve with the `ForwardFromCurve` model.

In [142]:
# Create market context
market_data_scope = "fx-forward-pricing-specified-forward-rate"
market_context = create_market_context(
    scope=market_data_scope,
    market_rules=[
        # Market data rule for resolving FX spot rates
        lm.MarketDataKeyRule(
            key="Fx.*.*",
            data_scope=market_data_scope,
            supplier=market_supplier,
            quote_type="Rate",
            field="mid",
            quote_interval="5D.0D",
        ),
        # Market data rule for resolving forward rate quote
        lm.MarketDataKeyRule(
            key="Quote.LusidInstrumentId.*",
            data_scope=market_data_scope,
            supplier=market_supplier,
            quote_type="Price",
            field="mid",
        ),
        # Market data rule for resolving discount factor curves
        lm.MarketDataKeyRule(
            key="Rates.GBP.GBPOIS",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
        ),
    ],
)

# Not split by legs
pricing_context = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardSpecifiedRate",
            instrument_type="FxForward",
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=False),
)

recipe_code = "forward-rate-no-split"
pricing_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    description="Forward specified rate",
    market=market_context,
    pricing=pricing_context,
)

upsert_recipe(pricing_recipe)


# Split by legs
pricing_context_split = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardSpecifiedRate",
            instrument_type="FxForward",
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=True),
)

recipe_code_split = "forward-rate-split"
pricing_recipe_split = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code_split,
    description="Forward specified rate, split by legs",
    market=market_context,
    pricing=pricing_context_split,
)

upsert_recipe(pricing_recipe_split)

Now we upsert a forward rate quote and a spot rate quote for GBPJPY to run the single line valuation.

In [143]:
fwd_rate_gbpjpy = 190.03
quote_fwd_gbpjpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY/FxFwdRate/20241203",
            instrument_id_type="LusidInstrumentId",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=fwd_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

spot_rate_gbpjpy = 191.11
quote_gbp_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_fwd_gbpjpy, quote_gbp_jpy])

'Quotes successfully loaded into LUSID. 2 quotes upserted.'

Now we upsert a GBP discount factor curve.

In [144]:
# DOM currency discount factor curve
# Build curve data
df_months = 6
rfr = 0.05
df_dates = [None] * (df_months - 1)
dfs = [None] * (df_months - 1)
for i in range(1, df_months):
    df_dates[i - 1] = (valuation_date + timedelta(days=i * 30)).isoformat()
    dfs[i - 1] = math.exp(-30 / 360 * rfr * i)


df_curve_data = lm.DiscountFactorCurveData(
    base_date=valuation_date.isoformat(),
    dates=df_dates,
    discount_factors=dfs,
    market_data_type="DiscountFactorCurveData",
)

df_curve_id = lm.ComplexMarketDataId(
    provider="Lusid",
    price_source="",
    effective_at=valuation_date.isoformat(),
    market_asset="GBP/GBPOIS",
)

df_curve_request = lm.UpsertComplexMarketDataRequest(
    market_data_id=df_curve_id, market_data=df_curve_data
)

upsert_complex_market_data(scope=market_data_scope, request=df_curve_request)

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

We now demonstrate the valuation explicitly, using the data from the economic definition of the FX forward and the market data provided above. The methodology is similar to the `ForwardSpecifiedRateUndiscounted` example above, except that we need to calculate the GBP discount factor from the discount factor curve created above. The first step is to estimate the discount factor in the domestic currency to calculate the present value of the domestic leg. This will be estimated using linear interpolation on the points of the discount curve.

The points on the discount factor curve we have upserted are at +30 day increments from the valuation date (Mon 14/10/24), so the maturity date falls between the first two points of the curve; the +30 and +60 day increments, which are Wed 13/11/24 and Fri 13/12/24 respectively. Thus there are 20 days from +30 to maturity and 10 days from maturity to +60, so the estimated discount factor will be the 1:2 weighted average of the +30 and +60 day discount factors.

The PV of the domestic leg is `-100,000 * D`, where `D` is the discount factor estimated from the discount factor curve. The PV of the foreign leg in the domestic currency GBP is given by `(19,000,000 / F) * D`, then the PV of the foreign leg in JPY is calculated by converting at the GBPJPY spot rate `S`, so is given by `(19,000,000 / F) * D * S`. The net PV in GBP is the sum of the PVs of the individual legs in GBP, so is `-100,000 * D + (19,000,000 / F) * D`.

In [145]:
def manual_valuation():
    # Estimate DOM discount factor using linear interpolation:
    rfr = 0.05
    df_1m = math.exp(-30 / 360 * rfr * 1)
    df_2m = math.exp(-30 / 360 * rfr * 2)
    plus30 = valuation_date + timedelta(days=30)
    plus60 = valuation_date + timedelta(days=60)
    d1 = (maturity_date - plus30).days
    d2 = (plus60 - maturity_date).days
    df_interpolated = (d2 * df_1m + d1 * df_2m) / (d1 + d2)
    display(f"Interpolated discount factor: {df_interpolated}")

    fwd_rate = 190.03
    dom_amt = -100_000
    fgn_amt = 19_000_000
    spot_rate = 191.11

    dom_leg_pv_gbp = dom_amt * df_interpolated
    fgn_leg_pv_gbp = (fgn_amt / fwd_rate) * df_interpolated
    net_pv_gbp = dom_leg_pv_gbp + fgn_leg_pv_gbp
    display(f"Domestic leg PV (GBP): {dom_leg_pv_gbp}")
    display(f"Foreign leg PV (GBP): {fgn_leg_pv_gbp}")
    display(f"Net PV (GBP): {net_pv_gbp}")

    fgn_leg_pv_jpy = fgn_leg_pv_gbp * spot_rate
    display(f"Foreign leg PV (JPY): {fgn_leg_pv_jpy}")


manual_valuation()

'Interpolated discount factor: 0.9930815290409539'

'Domestic leg PV (GBP): -99308.15290409539'

'Foreign leg PV (GBP): 99292.475144862'

'Net PV (GBP): -15.6777592333965'

'Foreign leg PV (JPY): 18975784.924934577'

#### Forward Specified Rate - No Split

In [146]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code,
)

valuation_df

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,None,-15.6777,GBP,-15.6777


#### Forward Specified Rate - Split By Legs

In [147]:
spot_rate_jpygbp = 1 / 191.11
quote_jpy_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="JPY/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_jpygbp, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_jpy_gbp])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In [148]:
valuation_df_split = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code_split,
)

valuation_df_split

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-99,307.9612",GBP,"-99,307.9612"
1,GBPJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"18,975,748.3035",JPY,"99,292.2835"


### 2.9 Discounting

The `Discounting` pricing model is not strictly a separate model, rather it unifies the `ForwardFromCurve`, `ForwardWithPoints` and `ForwardSpecifiedRate` models, both the discounted and undiscounted forms, allowing the pricer to be specified using the `Observable` and `DiscountingMethod` model options. In particular, pricing using the `ForwardFromCurve` model can be achieved using the `Discounting` model with `FxForwardCurve` observable type and `Standard` discounting method.

The complete mapping is shown in the table below:

| Observable Type | Discounting Method       | Equivalent Model                 |
| --------------- | ------------------------ | ----------------                 |
| FxForwardCurve  | Standard                 | ForwardFromCurve                 |
| FxForwardCurve  | ConstantTimeValueOfMoney | ForwardFromCurveUndiscounted     |
| ForwardPoints   | Standard                 | ForwardWithPoints                |
| ForwardPoints   | ConstantTimeValueOfMoney | ForwardWithPointsUndiscounted    |
| ForwardRate     | Standard                 | ForwardSpecifiedRate             |
| ForwardRate     | ConstantTimeValueOfMoney | ForwardSpecifiedRateUndiscounted |


**NOTE:** The `Discounting` pricing model can not be used to perform `SimpleStatic` pricing.

**NOTE:** It is technically possible to get back to the `ConstantTimeValueOfMoney` pricing model using `Discounting` with `Invalid` observable type and `ConstantTimeValueOfMoney` discounting method.

We now demonstrate an example using the `Discounting` model with `FxForwardCurve` observable type and `ConstantTimeValueOfMoney` discounting method to arrive at the same result as using the `ForwardFromCurveUndiscounted` pricing model.

In [149]:
# Create market context
market_data_scope = "discounting-forward-curve-ctvom"
market_context = create_market_context(
    scope=market_data_scope,
    market_rules=[
        # Market data rule for resolving FX spot rates
        lm.MarketDataKeyRule(
            key="Fx.*.*",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
            quote_interval="5D.0D",
        ),
        # Market data rule for resolving forward rates curves
        lm.MarketDataKeyRule(
            key="FxForwards.*.*.FxFwdCurve",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
            quote_interval="1Y.0D",
        ),
    ],
)

# Not split by legs
pricing_context = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="Discounting",
            instrument_type="FxForward",
            model_options=lm.FxForwardModelOptions(
                model_options_type="FxForwardModelOptions",
                convert_to_report_ccy=False,
                discounting_method="ConstantTimeValueOfMoney",
                forward_rate_observable_type="FxForwardCurve",
            ),
        )
    ]
)

recipe_code = "curve-undiscounted-no-split"
pricing_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    description="Discounting model, FX forward curve, no discounting",
    market=market_context,
    pricing=pricing_context,
)

upsert_recipe(pricing_recipe)


# Split by legs
pricing_context_split = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="Discounting",
            instrument_type="FxForward",
            model_options=lm.FxForwardModelOptions(
                model_options_type="FxForwardModelOptions",
                convert_to_report_ccy=False,
                discounting_method="ConstantTimeValueOfMoney",
                forward_rate_observable_type="FxForwardCurve",
            ),
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=True),
)

recipe_code_split = "curve-undiscounted-split"
pricing_recipe_split = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code_split,
    description="Discounting model, FX forward curve, no discounting, split by legs",
    market=market_context,
    pricing=pricing_context_split,
)

upsert_recipe(pricing_recipe_split)

Now we create and upsert a GBPJPY forward rates tenor curve and a GBPJPY spot rate quote.

In [150]:
# Upsert market data
# FX forward tenor curve
forward_curve_data = lm.FxForwardTenorCurveData(
    market_data_type="FxForwardTenorCurveData",
    base_date=valuation_date.isoformat(),
    dom_ccy=dom_ccy,
    fgn_ccy=fgn_ccy,
    tenors=["1W", "2W", "1M", "2M", "3M"],
    rates=[188.32, 188.14, 187.67, 186.93, 186.11],
)

fwd_curve_id = lm.ComplexMarketDataId(
    provider="Lusid",
    price_source=None,
    effective_at=valuation_date.isoformat(),
    market_asset="GBP/JPY/FxFwdCurve",
)

complex_market_data_request = lm.UpsertComplexMarketDataRequest(
    market_data_id=fwd_curve_id,
    market_data=forward_curve_data,
)

# Upsert forward curve into LUSID
response = upsert_complex_market_data(
    scope=market_data_scope, request=complex_market_data_request
)

# GBPJPY spot rate
spot_rate_gbpjpy = 188.5
quote_gbp_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_gbp_jpy])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

#### Discounting - No Split

The result below shows the valuation of the FX forward in LUSID using the `Discounting` pricing model, using a GBPJPY forward rates curve and a GBPJPY spot rate with no discounting. This is consistent with the result obtained using the same market data with the [ForwardFromCurveUndiscounted](#23-forward-from-curve-undiscounted) model, and the calculation is exactly as illustrated in this section.

In [151]:
# Not split by legs
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope, date=valuation_date, recipe_code=recipe_code
)

display(valuation_df)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,None,"1,455.8534",GBP,"1,455.8534"


#### Discounting - Split By Legs

As above, the methodology for calculating separate valuations for each leg with the `Discounting` pricing model using a GBPJPY forward rates curve with no discounting is described in the section [ForwardFromCurveUndiscounted](#23-forward-from-curve-undiscounted). To calculate the PV of the individual legs in the report currency (here `report_ccy = pf_ccy = "GBP"`), we also need a JPYGBP spot rate quote.

In [152]:
spot_rate_jpygbp = 0.00531
quote_jpy_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="JPY/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_jpygbp, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(scope=market_data_scope, quotes=[quote_jpy_gbp])

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In [153]:
valuation_split_df = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code_split,
)

display(valuation_split_df)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-100,000.0000",GBP,"-100,000.0000"
1,GBPJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"19,124,428.3573",JPY,"101,550.7146"


Observe that the valuation results using the `Discounting` model with `discounting_method="ConstantTimeValueOfMoney"` and `forward_rate_observable_type="FxForwardCurve"`, given the same market data, are identical to those obtained by using the [ForwardFromCurveUndiscounted](#23-forward-from-curve-undiscounted) model by name directly without model options.

## 3. Reporting Currency

There are two different ways to obtain a valuation in the reporting currency and it is important to note that they are not interchangeable.

The first way, which we have demonstrated in the examples above, is to include the valuation key `Valuation/PvInReportCcy` in the valuation request, while using the `convert_to_report_ccy=False` model option. With this configuration, the `Valuation/PV` will be given in the domestic currency of the forward (or the currency of each leg if pricing legs individually), while `Valuation/PvInReportCcy` is given in the reporting currency and is calculated using the forward and spot rates.

The second way, which we demonstrate below, is to set the `convert_to_report_ccy=True` model option and provide the `report_ccy` parameter in the valuation request. With this configuration, both the `Valuation/PV` and `Valuation/PvInReportCcy` are converted to the report currency using the forward rates for RPTDOM and RPTFGN currency pairs.

In the above examples, we have made a simplifying assumption that the domestic currency on the FX forward, the portfolio currency and the reporting currency for the valuation are all GBP, so there are only two relevant currencies.

We now demonstrate an example where the reporting currency for valuation and the two currencies in the FX forward are all distinct. For this example, we will use the existing portfolio (GBP), the existing GBPJPY forward and report in CAD. We will use the `ForwardFromCurveUndiscounted` model with the `convert_to_report_ccy=True` model option and the `produce_separate_result_for_linear_otc_legs=True` pricing option.

To show that the valuation genuinely depends on the forward rates from the curve and not on the spot rates, we now create a new FX forward with the same start date and foreign and domestic amounts, but with a different maturity. Then we will see that the two valuations differ, while if the valuation were using the spot rates, we would see no difference in the two valuations, since we are assuming no discounting.

First create and upsert the FX forward that only differs in maturity date.

In [154]:
forward_name = "GBPJPY 5M Forward"
forward_identifier = "FWD-GBPJPY5M"

start_date = datetime(2024, 6, 3, tzinfo=pytz.utc)
maturity_date = datetime(2024, 11, 4, tzinfo=pytz.utc)
dom_amount = -100_000
dom_ccy = "GBP"
fgn_amount = 19_000_000
fgn_ccy = "JPY"

# Create FX forward definition
fx_forward = lm.FxForward(
    start_date=start_date.isoformat(),
    maturity_date=maturity_date.isoformat(),
    dom_amount=dom_amount,
    dom_ccy=dom_ccy,
    fgn_amount=fgn_amount,
    fgn_ccy=fgn_ccy,
    instrument_type="FxForward",
)

# Create LUSID instrument definition
forward_definition = lm.InstrumentDefinition(
    name=forward_name,
    identifiers={"ClientInternal": lm.InstrumentIdValue(value=forward_identifier)},
    definition=fx_forward,
)

# Upsert FX forward instrument
try:
    upsert_response = instruments_api.upsert_instruments(
        request_body={forward_identifier: forward_definition}
    )
    luid = upsert_response.values[forward_identifier].lusid_instrument_id
    display(f"Success! Upserted instrument {luid}")
except KeyError as e:
    display(
        f"Failed to upsert instrument {forward_identifier}. Details: {upsert_response.failed[forward_identifier].detail}"
    )
except lusid.ApiException as e:
    body = json.loads(e.body)
    display(
        f"Title: {body['title']}, Details: {body['detail']}, Errors: {body['errors']}"
    )

'Success! Upserted instrument LUID_000185GQ'

Do a `StockIn` transaction for one unit of the FX forward.

In [155]:
# Trade variables
settle_days = 2
trade_date = start_date
settlement_date = start_date + timedelta(days=settle_days)

# Create StockIn transaction
stock_in_txn = lm.TransactionRequest(
    transaction_id="TXN002",
    type="StockIn",
    instrument_identifiers={"Instrument/default/ClientInternal": forward_identifier},
    transaction_date=trade_date.isoformat(),
    settlement_date=settlement_date.isoformat(),
    units=1,
    transaction_price=lm.TransactionPrice(price=0, type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=0, currency=pf_ccy),
    exchange_rate=1,
    transaction_currency=pf_ccy,
)

# Upsert StockIn transaction
try:
    response = transaction_portfolios_api.upsert_transactions(
        scope=pf_scope, code=pf_code, transaction_request=[stock_in_txn]
    )
    display(f"Transaction successfully updated at time {response.version.as_at_date}")

except lusid.ApiException as e:
    body = json.loads(e.body)
    display(
        f"Failed to upsert transaction - Title: {body['title']}, Details: {body['detail']}, Errors: {body['errors']}"
    )

'Transaction successfully updated at time 2024-09-27 12:25:10.143876+00:00'

Next, we create the pricing recipe with the `produce_separate_result_for_linear_otc_legs=True` and `convert_to_report_ccy=True` options.

In [156]:
# Create market context
market_data_scope = "fx-forward-report-currency"
market_context = create_market_context(
    scope=market_data_scope,
    market_rules=[
        # Market data rule for resolving FX spot rates
        lm.MarketDataKeyRule(
            key="Fx.*.*",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
            quote_interval="5D.0D",
        ),
        # Market data rule for resolving forward rates curves
        lm.MarketDataKeyRule(
            key="FxForwards.*.*.FxFwdCurve",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
            quote_interval="1Y.0D",
        ),
    ],
)

# Create pricing context
pricing_context = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardFromCurveUndiscounted",
            instrument_type="FxForward",
            model_options=lm.FxForwardModelOptions(
                model_options_type="FxForwardModelOptions",
                convert_to_report_ccy=True,
                discounting_method="Invalid",
                forward_rate_observable_type="FxForwardCurve",
            ),
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=True),
)

recipe_code = "curve-undiscounted-report-currency"
pricing_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    description="FX forward curve, no discounting, split by legs",
    market=market_context,
    pricing=pricing_context,
)

upsert_recipe(pricing_recipe)

Next, we upsert forward rate curves and spot rate quotes for each currency pair. We will need forward rate curves for CADGBP and CADJPY (In general, RPTDOM and RPTFGN curves are needed) and spot rates for CADGBP, GBPJPY and CADJPY.

In [157]:
# Upsert market data
# Note that although the market data resolver requires these spot rates to perform valuation,
# they are not used at all in the code path when converting to report currency.
spot_rate_cadgbp = 0.568143
quote_cad_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="CAD/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_cadgbp, unit="rate"),
    lineage="InternalSystem",
)

spot_rate_gbpjpy = 188.50
quote_gbp_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

spot_rate_cadjpy = 107.09
quote_cad_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="CAD/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_cadjpy, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(
    scope=market_data_scope, quotes=[quote_cad_gbp, quote_gbp_jpy, quote_cad_jpy]
)


# Upsert complex market data
## FX forward tenor curve CADJPY
forward_curve_data_cadjpy = lm.FxForwardTenorCurveData(
    market_data_type="FxForwardTenorCurveData",
    base_date=valuation_date.isoformat(),
    dom_ccy="CAD",
    fgn_ccy="JPY",
    tenors=["1W", "2W", "1M", "2M", "3M"],
    rates=[107.132, 107.131, 107.129, 107.125, 107.122],
)

fwd_curve_id_cadjpy = lm.ComplexMarketDataId(
    provider="Lusid",
    price_source=None,
    effective_at=valuation_date.isoformat(),
    market_asset="CAD/JPY/FxFwdCurve",
)

complex_market_data_request_cadjpy = lm.UpsertComplexMarketDataRequest(
    market_data_id=fwd_curve_id_cadjpy,
    market_data=forward_curve_data_cadjpy,
)

# Upsert forward curve into LUSID
response = upsert_complex_market_data(
    scope=market_data_scope, request=complex_market_data_request_cadjpy
)


## FX Forward tenor curve CADGBP
forward_curve_data_cadgbp = lm.FxForwardTenorCurveData(
    market_data_type="FxForwardTenorCurveData",
    base_date=valuation_date.isoformat(),
    dom_ccy="CAD",
    fgn_ccy="GBP",
    tenors=["1W", "2W", "1M", "2M", "3M"],
    rates=[0.568391, 0.568516, 0.568677, 0.569041, 0.569484],
)

fwd_curve_id_cadgbp = lm.ComplexMarketDataId(
    provider="Lusid",
    price_source=None,
    effective_at=valuation_date.isoformat(),
    market_asset="CAD/GBP/FxFwdCurve",
)

complex_market_data_request_cadgbp = lm.UpsertComplexMarketDataRequest(
    market_data_id=fwd_curve_id_cadgbp,
    market_data=forward_curve_data_cadgbp,
)

# Upsert forward curve into LUSID
response = upsert_complex_market_data(
    scope=market_data_scope, request=complex_market_data_request_cadgbp
)

'Quotes successfully loaded into LUSID. 3 quotes upserted.'

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

In [158]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code,
    report_currency="CAD",
)

display(valuation_df)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-175,786.4687",CAD,"-175,786.4687"
1,GBPJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"177,359.8191",CAD,"177,359.8191"
2,GBPJPY 5M Forward,2024-06-03,2024-11-04,DomesticLeg,"-175,883.4370",CAD,"-175,883.4370"
3,GBPJPY 5M Forward,2024-06-03,2024-11-04,ForeignLeg,"177,353.8317",CAD,"177,353.8317"


Note that currently the spot rates CADGBP, CADJPY and GBPJPY are requested when performing the valuation, but when using the `convert_to_report_ccy=True` option, their values are not used at all. To demonstrate this, we now update the spot rate quotes with some arbitrary values and observe that the valuation does not change at all.

In [159]:
spot_rate_cadgbp = 0.4  # 0.568143
quote_cad_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="CAD/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_cadgbp, unit="rate"),
    lineage="InternalSystem",
)

spot_rate_gbpjpy = 230  # 188.50
quote_gbp_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

spot_rate_cadjpy = 90  # 107.09
quote_cad_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="CAD/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_cadjpy, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(
    scope=market_data_scope, quotes=[quote_cad_gbp, quote_gbp_jpy, quote_cad_jpy]
)

'Quotes successfully loaded into LUSID. 3 quotes upserted.'

In [160]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code,
    report_currency="CAD",
)

display(valuation_df)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-175,786.4687",CAD,"-175,786.4687"
1,GBPJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"177,359.8191",CAD,"177,359.8191"
2,GBPJPY 5M Forward,2024-06-03,2024-11-04,DomesticLeg,"-175,883.4370",CAD,"-175,883.4370"
3,GBPJPY 5M Forward,2024-06-03,2024-11-04,ForeignLeg,"177,353.8317",CAD,"177,353.8317"


Summing the PVs of the domestic and foreign legs gives net valuations for the 6M and 5M forwards respectively as

In [161]:
net_pv_6m = 177_359.8191 - 175_786.4687
display(f"Net PV 6M: {net_pv_6m} CAD")

net_pv_5m = 177_353.8317 - 175_883.4370
display(f"Net PV 5M: {net_pv_5m} CAD")

'Net PV 6M: 1573.3503999999957 CAD'

'Net PV 5M: 1470.3947000000044 CAD'

Note that these two valuations are different since the calculation is based on 5M and 6M forward rates from the CADGBP and CADJPY forward curves, rather than the spot rates. Now we infer the forward rates used from the individual leg valuations and show how these can be determined using linear interpolation on the forward curves. For each leg, the inferred forward rate is simply the leg valuation in the report currency divided by the leg amount in the forward definition.

In [162]:
def inferred_forward_rates():
    fwd_cadgbp_5m = -100_000 / -175_883.4370
    fwd_cadgbp_6m = -100_000 / -175_786.4687
    fwd_cadjpy_5m = 19_000_000 / 177_353.8317
    fwd_cadjpy_6m = 19_000_000 / 177_359.8191

    display(f"Forward rate CADGBP 5M: {fwd_cadgbp_5m}")
    display(f"Forward rate CADGBP 6M: {fwd_cadgbp_6m}")
    display(f"Forward rate CADJPY 5M: {fwd_cadjpy_5m}")
    display(f"Forward rate CADJPY 6M: {fwd_cadjpy_6m}")


inferred_forward_rates()

'Forward rate CADGBP 5M: 0.5685583685745236'

'Forward rate CADGBP 6M: 0.5688719998731051'

'Forward rate CADJPY 5M: 107.13047368572866'

'Forward rate CADJPY 6M: 107.12685712251046'

We now derive the forward rates using linear interpolation on the forward curves and observe these are consistent with the forward rates inferred from the valuations.

The 5M maturity date (Mon 04/11/24) falls between the 2W and 1M tenors and the 6M maturity date (Tue 03/12/24) falls between the 1M and 2M tenors.

**2W:** 14 days after the spot date of Wed 16/10/24 is Wed 30/10/24. This is 5 days before the 5M maturity.

**1M:** Adding one month to the spot date (Wed 16/10/24) and rolling forward over the weekend gives the 1M tenor date as Mon 18/11/24. This is 14 days after the 5M maturity date and 15 days before the 6M maturity date.

**2M:** Adding one month to the spot date (Wed 16/10/24) gives the 2M tenor date as Mon 16/12/24. This is 13 days after the 6M maturity.

In [163]:
def curve_forward_rates():
    fwd_cadgbp_2w = 0.568516
    fwd_cadgbp_1m = 0.568677
    fwd_cadgbp_2m = 0.569041
    fwd_cadgbp_interpolated_5m = (14 * fwd_cadgbp_2w + 5 * fwd_cadgbp_1m) / 19
    display(
        f"Interpolated forward rate CADGBP 5M maturity: {fwd_cadgbp_interpolated_5m}"
    )
    fwd_cadgbp_interpolated_6m = (13 * fwd_cadgbp_1m + 15 * fwd_cadgbp_2m) / 28
    display(
        f"Interpolated forward rate CADGBP 6M maturity: {fwd_cadgbp_interpolated_6m}"
    )

    fwd_cadjpy_2w = 107.131
    fwd_cadjpy_1m = 107.129
    fwd_cadjpy_2m = 107.125
    fwd_cadjpy_interpolated_5m = (14 * fwd_cadjpy_2w + 5 * fwd_cadjpy_1m) / 19
    display(
        f"Interpolated forward rate CADJPY 5M maturity: {fwd_cadjpy_interpolated_5m}"
    )
    fwd_cadjpy_interpolated_6m = (13 * fwd_cadjpy_1m + 15 * fwd_cadjpy_2m) / 28
    display(
        f"Interpolated forward rate CADJPY 6M maturity: {fwd_cadjpy_interpolated_6m}"
    )


curve_forward_rates()

'Interpolated forward rate CADGBP 5M maturity: 0.5685583684210527'

'Interpolated forward rate CADGBP 6M maturity: 0.5688719999999999'

'Interpolated forward rate CADJPY 5M maturity: 107.13047368421053'

'Interpolated forward rate CADJPY 6M maturity: 107.12685714285715'

Now clean up and remove the 5M forward from the portfolio.

In [164]:
# Cleanup
stock_in_txn = lm.TransactionRequest(
    transaction_id="TXN002",
    type="StockIn",
    instrument_identifiers={"Instrument/default/ClientInternal": forward_identifier},
    transaction_date=trade_date.isoformat(),
    settlement_date=settlement_date.isoformat(),
    units=0,
    transaction_price=lm.TransactionPrice(price=0, type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=0, currency=pf_ccy),
    exchange_rate=1,
    transaction_currency=pf_ccy,
)

# Upsert StockIn transaction
try:
    response = transaction_portfolios_api.upsert_transactions(
        scope=pf_scope, code=pf_code, transaction_request=[stock_in_txn]
    )
    display(f"Transaction successfully updated at time {response.version.as_at_date}")

except lusid.ApiException as e:
    body = json.loads(e.body)
    display(
        f"Failed to upsert transaction - Title: {body['title']}, Details: {body['detail']}, Errors: {body['errors']}"
    )

'Transaction successfully updated at time 2024-09-27 12:25:12.516610+00:00'

## 4. Forward Rates and Undiscounted Models

When valuing a portfolio with two FX forwards with the same maturities and the same values for `fgn_ccy` and `fgn_amount` using an undiscounted model, such as `ForwardFromCurveUndiscounted`, together with the `convert_to_report_ccy=False` model option, the PVs of the foreign legs in the foreign currency may differ between the two forwards. The example below shows a GBPJPY forward and a USDJPY forward with the same JPY leg amount and we will see that the valuations of the JPY legs are different for each forward. This occurs because we use forward rates to calculate the leg PVs in `dom_ccy`, then the PV of the foreign leg in `fgn_ccy` is calculated using a spot rate DOMFGN. 

In [165]:
forward_name = "USDJPY 6M Forward"
forward_identifier = "FWD-USDJPY6M"

start_date = datetime(2024, 6, 3, tzinfo=pytz.utc)
maturity_date = datetime(2024, 12, 3, tzinfo=pytz.utc)
dom_amount = -126_666
dom_ccy = "USD"
fgn_amount = 19_000_000
fgn_ccy = "JPY"

# Create FX forward definition
fx_forward = lm.FxForward(
    start_date=start_date.isoformat(),
    maturity_date=maturity_date.isoformat(),
    dom_amount=dom_amount,
    dom_ccy=dom_ccy,
    fgn_amount=fgn_amount,
    fgn_ccy=fgn_ccy,
    instrument_type="FxForward",
)

# Create LUSID instrument definition
forward_definition = lm.InstrumentDefinition(
    name=forward_name,
    identifiers={"ClientInternal": lm.InstrumentIdValue(value=forward_identifier)},
    definition=fx_forward,
)

# Upsert FX forward instrument
try:
    upsert_response = instruments_api.upsert_instruments(
        request_body={forward_identifier: forward_definition}
    )
    luid = upsert_response.values[forward_identifier].lusid_instrument_id
    display(f"Success! Upserted instrument {luid}")
except KeyError as e:
    display(
        f"Failed to upsert instrument {forward_identifier}. Details: {upsert_response.failed[forward_identifier].detail}"
    )
except lusid.ApiException as e:
    body = json.loads(e.body)
    display(
        f"Title: {body['title']}, Details: {body['detail']}, Errors: {body['errors']}"
    )

'Success! Upserted instrument LUID_000185GO'

Having created and upserted the USDJPY FX forward, we now add a `StockIn` transaction to create a position of one unit of the forward without incurring any costs which would affect the cash holding.

In [166]:
# Trade variables
settle_days = 2
trade_date = start_date
settlement_date = start_date + timedelta(days=settle_days)

# Create StockIn transaction
stock_in_txn = lm.TransactionRequest(
    transaction_id="TXN003",
    type="StockIn",
    instrument_identifiers={"Instrument/default/ClientInternal": forward_identifier},
    transaction_date=trade_date.isoformat(),
    settlement_date=settlement_date.isoformat(),
    units=1,
    transaction_price=lm.TransactionPrice(price=0, type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=0, currency=pf_ccy),
    exchange_rate=1,
    transaction_currency=pf_ccy,
)

# Upsert StockIn transaction
try:
    response = transaction_portfolios_api.upsert_transactions(
        scope=pf_scope, code=pf_code, transaction_request=[stock_in_txn]
    )
    display(f"Transaction successfully updated at time {response.version.as_at_date}")

except lusid.ApiException as e:
    body = json.loads(e.body)
    display(
        f"Failed to upsert transaction - Title: {body['title']}, Details: {body['detail']}, Errors: {body['errors']}"
    )

'Transaction successfully updated at time 2024-09-27 12:25:13.268462+00:00'

Now create the pricing recipes using the `convert_to_report_ccy=False` option.

In [167]:
# Create market context
market_data_scope = "fx-forward-undiscounted-leg-level"
market_context = create_market_context(
    scope=market_data_scope,
    market_rules=[
        # Market data rule for resolving FX spot rates
        lm.MarketDataKeyRule(
            key="Fx.*.*",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
            quote_interval="5D.0D",
        ),
        # Market data rule for resolving forward rates curves
        lm.MarketDataKeyRule(
            key="FxForwards.*.*.FxFwdCurve",
            supplier="Lusid",
            data_scope=market_data_scope,
            quote_type="Rate",
            field="mid",
            quote_interval="1Y.0D",
        ),
    ],
)

# Create pricing context
pricing_context = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardFromCurveUndiscounted",
            instrument_type="FxForward",
            model_options=lm.FxForwardModelOptions(
                model_options_type="FxForwardModelOptions",
                convert_to_report_ccy=False,
                discounting_method="Invalid",
                forward_rate_observable_type="FxForwardCurve",
            ),
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=False),
)

recipe_code = "curve-undiscounted-leg-level"
pricing_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    description="FX forward curve, no discounting",
    market=market_context,
    pricing=pricing_context,
)

upsert_recipe(pricing_recipe)

pricing_context_split = lm.PricingContext(
    model_rules=[
        lm.VendorModelRule(
            supplier=market_supplier,
            model_name="ForwardFromCurveUndiscounted",
            instrument_type="FxForward",
            model_options=lm.FxForwardModelOptions(
                model_options_type="FxForwardModelOptions",
                convert_to_report_ccy=False,
                discounting_method="Invalid",
                forward_rate_observable_type="FxForwardCurve",
            ),
        )
    ],
    options=lm.PricingOptions(produce_separate_result_for_linear_otc_legs=True),
)

recipe_code_split = "curve-undiscounted-leg-level-split"
pricing_recipe_split = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code_split,
    description="FX forward curve, no discounting, split by legs",
    market=market_context,
    pricing=pricing_context_split,
)

upsert_recipe(pricing_recipe_split)

Next, we upsert spot rate quotes and forward rate tenor curves. We will need a GBPJPY forward rates curve for the GBPJPY forward and a USDJPY forward rate curve for the USDJPY forward. For the GBPJPY forward, we will also need spot rate quotes for GBPJPY and JPYGBP. For the USDJPY forward, we will need a USDJPY spot rate and a USDGBP spot rate to convert to the portfolio currency.

In [168]:
# Upsert market data
spot_rate_gbpjpy = 188.5
quote_gbp_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="GBP/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_gbpjpy, unit="rate"),
    lineage="InternalSystem",
)

spot_rate_jpygbp = 1 / 188.5
quote_jpy_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="JPY/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_jpygbp, unit="rate"),
    lineage="InternalSystem",
)

spot_rate_usdjpy = 146.5
quote_usd_jpy = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="USD/JPY",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_usdjpy, unit="rate"),
    lineage="InternalSystem",
)

spot_rate_usdgbp = spot_rate_usdjpy / spot_rate_gbpjpy
quote_usd_gbp = lm.UpsertQuoteRequest(
    quote_id=lm.QuoteId(
        quote_series_id=lm.QuoteSeriesId(
            provider="Lusid",
            instrument_id="USD/GBP",
            instrument_id_type="CurrencyPair",
            quote_type="Price",
            field="mid",
        ),
        effective_at=valuation_date.isoformat(),
    ),
    metric_value=lm.MetricValue(value=spot_rate_usdgbp, unit="rate"),
    lineage="InternalSystem",
)

upsert_quotes(
    scope=market_data_scope,
    quotes=[quote_gbp_jpy, quote_jpy_gbp, quote_usd_jpy, quote_usd_gbp],
)


# FX forward tenor curve GBPJPY
forward_curve_data = lm.FxForwardTenorCurveData(
    market_data_type="FxForwardTenorCurveData",
    base_date=valuation_date.isoformat(),
    dom_ccy="GBP",
    fgn_ccy="JPY",
    tenors=["1W", "2W", "1M", "2M", "3M"],
    rates=[188.32, 188.14, 187.67, 186.93, 186.11],
)

fwd_curve_id = lm.ComplexMarketDataId(
    provider="Lusid",
    price_source=None,
    effective_at=valuation_date.isoformat(),
    market_asset="GBP/JPY/FxFwdCurve",
)

complex_market_data_request = lm.UpsertComplexMarketDataRequest(
    market_data_id=fwd_curve_id,
    market_data=forward_curve_data,
)

# Upsert forward curve into LUSID
response = upsert_complex_market_data(
    scope=market_data_scope, request=complex_market_data_request
)


# FX forward tenor curve USDJPY
forward_curve_data = lm.FxForwardTenorCurveData(
    market_data_type="FxForwardTenorCurveData",
    base_date=valuation_date.isoformat(),
    dom_ccy="USD",
    fgn_ccy="JPY",
    tenors=["1W", "2W", "1M", "2M", "3M"],
    rates=[146.35, 146.19, 145.77, 145.21, 144.60],
)

fwd_curve_id = lm.ComplexMarketDataId(
    provider="Lusid",
    price_source=None,
    effective_at=valuation_date.isoformat(),
    market_asset="USD/JPY/FxFwdCurve",
)

complex_market_data_request = lm.UpsertComplexMarketDataRequest(
    market_data_id=fwd_curve_id,
    market_data=forward_curve_data,
)

# Upsert forward curve into LUSID
response = upsert_complex_market_data(
    scope=market_data_scope, request=complex_market_data_request
)

'Quotes successfully loaded into LUSID. 4 quotes upserted.'

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

'Quotes successfully loaded into LUSID. 1 quotes upserted.'

Now, we value the portfolio with both recipes, producing a net valuation for each forward, then a valuation for each individual leg of the forwards.

In [169]:
valuation_df = get_daily_valuation(
    market_data_scope=market_data_scope, date=valuation_date, recipe_code=recipe_code
)

display(valuation_df)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,None,"1,455.8534",GBP,"1,455.8534"
1,USDJPY 6M Forward,2024-06-03,2024-12-03,None,"3,945.1226",USD,"3,066.1032"


In [170]:
valuation_df_split = get_daily_valuation(
    market_data_scope=market_data_scope,
    date=valuation_date,
    recipe_code=recipe_code_split,
)

display(valuation_df_split)

,Instrument/CoreData/Name,Instrument/CoreData/StartDate,Instrument/CoreData/MaturityDate,Valuation/LegIdentifier,Valuation/PV,Valuation/PV/Ccy,Valuation/PvInReportCcy
0,GBPJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-100,000.0000",GBP,"-100,000.0000"
1,GBPJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"19,124,428.3573",JPY,"101,455.8534"
2,USDJPY 6M Forward,2024-06-03,2024-12-03,DomesticLeg,"-126,666.0000",USD,"-98,443.3369"
3,USDJPY 6M Forward,2024-06-03,2024-12-03,ForeignLeg,"19,134,529.4562",JPY,"101,509.4401"


Note that the present values of the JPY legs do not have the same value, though we may expect that in an undiscounted world both would still be the original foreign amount of 19,000,000 JPY. The reason for this difference, and the difference between the GBPJPY foreign leg and the USDJPY foreign leg, is that the foreign leg price is calculated as
`fgn_leg_price = (fgn_amount / fwd_rate) * spot_rate`.

In this example, `fgn_leg_price = 19,124,428.3573`, `spot_rate = 188.5` and the forward rate will be estimated from the curve below using linear interpolation. The maturity date (Tue 03/12/24) falls between the 1M and 2M tenor dates.

**1M:** Adding one month to the spot date (Wed 16/10/24) and rolling forward over the weekend gives the 1M tenor date as Mon 18/11/24. This is 15 days before the 6M maturity date.

**2M:** Adding one month to the spot date (Wed 16/10/24) gives the 2M tenor date as Mon 16/12/24. This is 13 days after the 6M maturity.

The period between the 1M and 2M tenors spans 28 days. The forward rate on the valuation date (Mon 14/10/24) implied by linear interpolation on the curve between the 1M and 2M tenors is: `f = f(1M) + (15/28) * (f(2M) - f(1M))`, which can be rearranged as `f = ((28 - 15)/28) * f(1M) + (15/28) * f(2M) = 13 * f(1M) + 15 * f(2M)`.

In [171]:
def demo_leg_level_pv():
    # Calculate forward rate from curve by interpolation
    gbpjpy_1m = 187.67
    gbpjpy_2m = 186.93
    spot_gbpjpy = 188.5
    interpolated_fwd_rate = (13 * gbpjpy_1m + 15 * gbpjpy_2m) / 28
    display(f"Interpolated GBPJPY forward rate: {interpolated_fwd_rate}")

    fgn_leg_gbp = 19_000_000 / interpolated_fwd_rate
    display(f"Foreign leg PV in GBP: {fgn_leg_gbp}")

    fgn_leg_jpy = fgn_leg_gbp * spot_gbpjpy
    display(f"Foreign leg PV in JPY: {fgn_leg_jpy}")


demo_leg_level_pv()

'Interpolated GBPJPY forward rate: 187.27357142857142'

'Foreign leg PV in GBP: 101455.85335433648'

'Foreign leg PV in JPY: 19124428.357292425'

Finally, clean up and remove the USDJPY forward from the portfolio.

In [172]:
# Create StockIn transaction
stock_in_txn = lm.TransactionRequest(
    transaction_id="TXN003",
    type="StockIn",
    instrument_identifiers={"Instrument/default/ClientInternal": forward_identifier},
    transaction_date=trade_date.isoformat(),
    settlement_date=settlement_date.isoformat(),
    units=0,
    transaction_price=lm.TransactionPrice(price=0, type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=0, currency=pf_ccy),
    exchange_rate=1,
    transaction_currency=pf_ccy,
)

# Upsert StockIn transaction
try:
    response = transaction_portfolios_api.upsert_transactions(
        scope=pf_scope, code=pf_code, transaction_request=[stock_in_txn]
    )
    display(f"Transaction successfully updated at time {response.version.as_at_date}")

except lusid.ApiException as e:
    body = json.loads(e.body)
    display(
        f"Failed to upsert transaction - Title: {body['title']}, Details: {body['detail']}, Errors: {body['errors']}"
    )

'Transaction successfully updated at time 2024-09-27 12:25:16.022790+00:00'

## 5. Interpolation on Forward Curves

Forward rate curves or forward points curves are defined by a list of dates or tenors together with a list of corresponding rates or forward points. For curves created based on tenors, using `FxForwardTenorCurveData`, the tenors are internally converted to dates. Rates or points for dates not in the given curve data are inferred using linear interpolation and extrapolation. For points between the earliest and the latest date, linear interpolation is used based on the latest date on the curve before the current date and the earliest date on the curve after the current date. For points lying outside this range, extrapolation is used and different extrapolation methods can be used for the 'front' and the 'back' of a curve.

The _front_ of a curve is the region up to the earliest date on the curve and the _back_ of a curve is the region after the latest date on the curve. The `ExtrapolationType` can be specified independently for the back and the front of the curve, as shown in the code snippet below. The available extrapolation types are: `None`, `Flat` and `Linear`. The front and back extrapolation types default to `Flat` if not specified.

- **ExtrapolationType.Flat (Default):** Points outside the interpolation region are given the value of the closest point on the curve within the interpolation region. If the front extrapolation type is `Flat`, points before the first date of the curve have the same value as the first date on the curve. If the back extrapolation type is flat, points after the last date on the curve are given the value at the last date on the curve.
- **ExtrapolationType.Linear:** Points outside the interpolation region are given by extending linearly using the last two data points at the boundary.
- **ExtrapolationType.None:** An error is thrown if extrapolation is attempted. That is, if points before the first date on the curve or after the last date on the curve are requested.

In [173]:
dates = [
    datetime(2024, 8, 1, tzinfo=pytz.utc).isoformat(),  # Thu 01 August 2024
    datetime(2024, 8, 15, tzinfo=pytz.utc).isoformat(),  # Thu 15 August 2024
    datetime(2024, 8, 29, tzinfo=pytz.utc).isoformat(),  # Thu 29 August 2024
]

rates = [192.4610, 188.8650, 190.5450]

forward_curve_data = lm.FxForwardCurveData(
    market_data_type="FxForwardCurveData",
    base_date=valuation_date.isoformat(),
    dom_ccy="GBP",
    fgn_ccy="JPY",
    dates=dates,
    rates=rates,
    market_data_options=lm.CurveOptions(
        market_data_options_type="CurveOptions",
        front_extrapolation_type="Flat",
        back_extrapolation_type="Linear",
    ),
)

In the example above, we have set `front_extrapolation_type="Flat"` and `back_extrapolation_type="Linear"` and the first point on the curve is (01/08/24, 192.4610) and the last point is (29/08/24, 190.5450). This means that dates before Thu 01/08/24 are given the boundary value 192.4610 and the values for dates after Thu 29/08/24 are given by extending linearly using the last two points on the curve `r1=(15/08/24, 188.8650)` and `r2=(29/08/24, 190.5450)`. The value given for Thu 05/09/24 by linear extrapolation is `r = 190.5450 + 7 * (190.5450 - 188.8650) / 14 = 191.3850`.

The linear interpolation methodology is standard. Suppose we are given are forward rates curve specified using dates and we are given a date `d` between the earliest and the latest dates on the curve. Let `d1` be the latest date on the curve before `d` and let `d2` be the earliest date on the curve after `d`, so `d1` and `d2` are the closest points on the curve to `d`. Let `a` be the number of calendar days from `d1` to `d` (Thursday to Thursday counts as 7 days) and let `b` be the number of calendar days from `d` to `d2`. The total number of days in the period `(d1, d2)` is `a + b`. Then the value of the curve at `d` given by linear interpolation is `r(d) = r(d1) + (a / (a + b)) * (r(d2) - r(d1))` or, equivalently `r(d) = (b / (a + b)) * r(d1) + (a / (a + b)) * r(d2)`, where `r(d)` denotes the value of the curve at `d`.

For example, if `d` is Fri 16/08/24, then the closest points on the curve are Thu 15/08/24 (`d1`) and Thu 29/08/24 (`d2`), with values `188.8650` and `190.5450` respectively. There is 1 day from `d1` to `d` and 13 days from `d` to `d2`, with 14 days from `d1` to `d2`. The value of the curve for Fri 16/08/24 is therefore `r = (13 / 14) * 188.8650 + (1 / 14) * 190.5450 = 188.985`.